# Flatiron Health mPC: Data Wrangling Test Set

**OBJECTIVE: Create a dataframe of relevant variables using test cohort patients which will be used to validate machine learning survival models.**

**BACKGROUND: The 12 CSV Flatiron files will be cleaned in the exact same fashion for the test set patients as for the training set patients. For more information on the cleaning process refer to Notebook: Data Wrangling Training Set.**

**OUTLINE:**
1. **File cleaning for patients in training set**
2. **Merge files to create master test dataframe** 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt

In [2]:
# Function that returns number of rows and count of unique PatientIDs for a dataframe. 
def row_ID(dataframe):
    row = dataframe.shape[0]
    ID = dataframe['PatientID'].nunique()
    return row, ID

In [3]:
#Import test IDs saved from Data Wrangling Training Set file. 
test_IDs = pd.read_csv('test_IDs.csv')

In [4]:
# Array of PatientIDs in training set.
test_IDs = test_IDs['PatientID'].to_numpy()

In [5]:
len(test_IDs)

3786

## Part 1: Data wrangling

**Relevant CSV files will be imported and processed. A file is considered processed when each row corresponds to a unique patient from the training set and each column is a relevant variable for mortality prognositication. The eligibility window for collecting variables is typically defined as -90 days and +30 days from index date. The index date is time of metastatic diagnosis. Plus 30 was selected as the upper bound of the eligibility window given that median time to start of first line treatment is about 30 days from metastatic diagnosis.** 

**The following CSV files from Flatiron will be cleaned:**
1. **Demographics and Practice**
2. **Enhanced_MetProstate**
3. **Enhanced_Mortality_V2**
4. **MedicationAdministration**
5. **Enhanced_MetPC_Biomarkers**
6. **Insurance**
7. **ECOG**
8. **Vitals**
9. **Labs**
10. **Diagnosis**
11. **Enhanced_MetPC_PrimaryTreatment**

### 1. Demographics

In [6]:
demographics = pd.read_csv('Demographics.csv')

In [7]:
demographics = demographics[demographics['PatientID'].isin(test_IDs)]

In [8]:
demographics.sample(5)

,PatientID,BirthYear,Gender,RaceSupplementedWithEthnicity,RaceEthnicitySource
16518,F639572B0CD2B,1948,M,White,Race
7682,F955F487FB53B,1937,M,White,Race
1997,FBF792EE74EC9,1942,M,Other Race,Race
8475,FA02CDAEA749B,1937,M,White,Race
14351,FC2D87430146B,1948,M,Other Race,Race


In [9]:
demographics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3786 entries, 1 to 18924
Data columns (total 5 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   PatientID                      3786 non-null   object
 1   BirthYear                      3786 non-null   int64 
 2   Gender                         3786 non-null   object
 3   RaceSupplementedWithEthnicity  3470 non-null   object
 4   RaceEthnicitySource            3786 non-null   object
dtypes: int64(1), object(4)
memory usage: 177.5+ KB


In [10]:
row_ID(demographics)

(3786, 3786)

#### Race and Ethnicity

In [11]:
conditions = [
    (demographics['RaceSupplementedWithEthnicity'] == 'White'),
    (demographics['RaceSupplementedWithEthnicity'] == 'Other Race'),
    (demographics['RaceSupplementedWithEthnicity'] == 'Black or African American'),
    (demographics['RaceSupplementedWithEthnicity'] == 'Asian')]

choices = ['white', 'other', 'black', 'asian']

demographics.loc[:, 'race'] = np.select(conditions, choices, default = 'unknown')

In [12]:
conditions = [
    (demographics['RaceSupplementedWithEthnicity'] == 'Hispanic or Latino'),
    (demographics['RaceSupplementedWithEthnicity'] == 'Not Hispanic or Latino')]

choices = ['hispanic_latino', 'not_hispanic_latino']

demographics.loc[:, 'ethnicity'] = np.select(conditions, choices, default = 'unknown')

In [13]:
demographics = demographics.drop(columns = ['RaceSupplementedWithEthnicity', 'RaceEthnicitySource'])

#### BirthYear

In [14]:
enhanced_met = pd.read_csv('Enhanced_MetProstate.csv')

In [15]:
enhanced_met = enhanced_met[enhanced_met['PatientID'].isin(test_IDs)]

In [16]:
demographics = pd.merge(demographics, enhanced_met[['PatientID', 'MetDiagnosisDate']], on = 'PatientID')

In [17]:
demographics.loc[:, 'MetDiagnosisDate'] = pd.to_datetime(demographics['MetDiagnosisDate'])

In [18]:
demographics.loc[:, 'age'] = demographics['MetDiagnosisDate'].dt.year - demographics['BirthYear']

In [19]:
demographics = demographics.drop(columns = ['BirthYear', 'MetDiagnosisDate'])

#### PracticeType

In [20]:
practice = pd.read_csv('Practice.csv')

In [21]:
practice = practice[practice['PatientID'].isin(test_IDs)]

In [22]:
practice_unique_count = (
    practice.groupby('PatientID')['PracticeType'].agg('nunique')
    .to_frame()
    .reset_index()
    .rename(columns = {'PracticeType': 'n_type'})
)

In [23]:
practice_n = pd.merge(practice, practice_unique_count, on = 'PatientID')

In [24]:
practice_n['p_type'] = (
    np.where(practice_n['n_type'] == 1, practice_n['PracticeType'], 'BOTH')
)

In [25]:
practice_n = (
    practice_n.drop_duplicates(subset = ['PatientID'], keep = 'first')
    .filter(items = ['PatientID', 'p_type'])
)

In [26]:
demographics = pd.merge(demographics, practice_n, on = 'PatientID')

In [27]:
row_ID(demographics)

(3786, 3786)

In [28]:
# Final training demographics table.
demographics.sample(5)

,PatientID,Gender,race,ethnicity,age,p_type
1468,FE53AA1FBE9E0,M,white,unknown,82,COMMUNITY
401,F541FA5DDA8FD,M,white,unknown,80,ACADEMIC
16,F958C61711495,M,white,unknown,82,ACADEMIC
2098,F6AF21A25B049,M,white,unknown,70,COMMUNITY
3380,F93D50CDBBA65,M,white,unknown,78,COMMUNITY


In [29]:
%whos DataFrame

Variable                Type         Data/Info
----------------------------------------------
demographics            DataFrame              PatientID Gende<...>\n[3786 rows x 6 columns]
enhanced_met            DataFrame               PatientID Diag<...>n[3786 rows x 13 columns]
practice                DataFrame               PatientID Prac<...>\n[3884 rows x 2 columns]
practice_n              DataFrame              PatientID     p<...>\n[3786 rows x 2 columns]
practice_unique_count   DataFrame              PatientID  n_ty<...>\n[3786 rows x 2 columns]


In [30]:
# Keep demographics and enhanced_met
del practice
del practice_n
del practice_unique_count

### 2. Enhanced_MetProstate

#### GroupStage 

In [31]:
# Dictionary for regrouping stages
stage_dict = { 
    'I': 'I',
    'II': 'II',
    'IIA': 'II',
    'IIB': 'II',
    'IIC': 'II',
    'III': 'III',
    'IIIA': 'III',
    'IIIB': 'III',
    'IIIC': 'III',
    'IV': 'IV',
    'IVA': 'IV',
    'IVB': 'IV',
    'Unknown / Not documented': 'unknown'
}

enhanced_met['stage'] = enhanced_met['GroupStage'].map(stage_dict)

In [32]:
enhanced_met = enhanced_met.drop(columns = ['GroupStage'])

#### TStage

In [33]:
enhanced_met = enhanced_met.drop(columns = ['TStage'])

#### NStage

**No modifications made to variable**

#### MStage

**No modifications made to variable**

#### MetDiagnosisDate

In [34]:
enhanced_met = enhanced_met.rename(columns = {'MetDiagnosisDate': 'met_date'})

In [35]:
enhanced_met.loc[:, 'met_date'] = pd.to_datetime(enhanced_met['met_date'])

In [36]:
enhanced_met.loc[:, 'met_year'] = enhanced_met['met_date'].dt.year

#### DiagnosisDate

In [37]:
enhanced_met = enhanced_met.rename(columns = {'DiagnosisDate': 'diagnosis_date'})

In [38]:
# Missing diagnosis_date will be replaced with met_date; other dates will be left untouched. 
enhanced_met['diagnosis_date'] = (
    np.where(enhanced_met['diagnosis_date'].isna(), enhanced_met['met_date'], enhanced_met['diagnosis_date'])
)

In [39]:
enhanced_met['diagnosis_date'] = pd.to_datetime(enhanced_met['diagnosis_date'])

#### Time from diagnosis date to metastatic date

In [40]:
enhanced_met.loc[:, 'delta_met_diagnosis'] = (enhanced_met['met_date'] - enhanced_met['diagnosis_date']).dt.days

#### CRPC

In [41]:
enhanced_met['CRPCDate'] = pd.to_datetime(enhanced_met['CRPCDate'])

In [42]:
# Make CRPC true if date of CRPC is <= to 30 days from metastatic diagnosis date. 
enhanced_met['crpc'] = (
    np.where(((enhanced_met['CRPCDate'] - enhanced_met['met_date']).dt.days <= 30), 1, 0)
)

In [43]:
# Make CRPC true if date of CRPC is <= to 30 days from metastatic diagnosis date. 
enhanced_met['crpc_time'] = (
    np.where(enhanced_met['crpc'] == 1, (enhanced_met['CRPCDate'] - enhanced_met['diagnosis_date']).dt.days, 0)
)

In [44]:
# Keep CRPCDate for mortality section.
enhanced_met = enhanced_met.drop(columns = ['IsCRPC'])

#### PSA 

In [45]:
enhanced_met['psa_diag_na'] = (
    np.where(enhanced_met['PSADiagnosis'].isna(), 1, 0)
)

In [46]:
enhanced_met['psa_met_na'] = (
    np.where(enhanced_met['PSAMetDiagnosis'].isna(), 1, 0)
)

In [47]:
row_ID(enhanced_met)

(3786, 3786)

In [48]:
# Final enhanced_met dataframe
enhanced_met.sample(5)

,PatientID,diagnosis_date,met_date,NStage,MStage,Histology,CRPCDate,GleasonScore,PSADiagnosis,PSAMetDiagnosis,stage,met_year,delta_met_diagnosis,crpc,crpc_time,psa_diag_na,psa_met_na
9665,F2856A2C922EC,1999-10-01,2019-03-07,Unknown / Not documented,M0,Adenocarcinoma,2015-06-30,10,NaN,6.88,unknown,2019,7097,1,5751.0,1,0
7122,F0AA24104D2C0,1996-01-01,2014-02-18,Unknown / Not documented,M0,Adenocarcinoma,2014-11-07,Unknown / Not documented,NaN,4.00,unknown,2014,6623,0,0.0,1,0
7921,FDF6E7F24F435,2006-04-07,2022-04-20,Unknown / Not documented,M0,Adenocarcinoma,2022-01-01,3 + 4 = 7,5.7,11.50,II,2022,5857,1,5748.0,0,0
1636,F36175137033C,2018-01-22,2018-01-22,N1,M1,Adenocarcinoma,NaT,9,959.0,959.00,IV,2018,0,0,0.0,0,0
7106,FA8D2D8A433E6,1996-01-01,2013-12-13,Unknown / Not documented,Unknown / Not documented,Adenocarcinoma,2014-04-08,7 (when breakdown not available),NaN,180.00,unknown,2013,6556,0,0.0,1,0


In [49]:
# Keep demographics and enhanced_met. 
%whos DataFrame

Variable       Type         Data/Info
-------------------------------------
demographics   DataFrame              PatientID Gende<...>\n[3786 rows x 6 columns]
enhanced_met   DataFrame               PatientID diag<...>n[3786 rows x 17 columns]


### 3. Enhanced_Mortality_V2

In [50]:
mortality = pd.read_csv('Enhanced_Mortality_V2.csv')

In [51]:
mortality = mortality[mortality['PatientID'].isin(test_IDs)]

In [52]:
row_ID(mortality)

(1958, 1958)

In [53]:
mortality = mortality.rename(columns = {'DateOfDeath': 'death_date'})

In [54]:
# For patients with year granularity, impute middle of the year (ie., July 1)
mortality['death_date'] = (
    np.where(mortality['death_date'].str.len() == 4, mortality['death_date'] + '-07-01', mortality['death_date'])
)

In [55]:
# For patients with month granularity, impute 15th of the month.
mortality['death_date'] = (
    np.where(mortality['death_date'].str.len() == 7, mortality['death_date'] + '-15', mortality['death_date'])
)

In [56]:
mortality['death_date'] = pd.to_datetime(mortality['death_date'])

#### Censoring

**For patients for whom a date of death is not known, the censor date can be defined either as the data cutoff date or as the last confirmed activity date. The last confirmed activity date is broadly defined as the last date at which there is evidence in the EHR that a patient is alive. Evidence of a record in at least one of the items listed below qualifies as patient-level confirmed activity:**
* **Visit: VisitDate**
* **Telemedicine: VisitDate**
* **Enhanced_MetPC_Orals: StartDate or EndDate**
* **Enhanced_MetPC_Biomarkers: SpecimenCollectedDate**
* **Enhanced_MetPC_ADT: StartDate or EndDate**
* **Enhanced_MetPC_AlphaBetaEmitters: AdministrationDate**
* **Enhanced_MetPC_PrimaryTreatment: TreatmentDate**
* **Enhanced_MetPC_Provenge: StartDate**
* **Enhanced_MetProstate: CRPCDate**

In [57]:
visit = pd.read_csv('Visit.csv')
telemedicine = pd.read_csv('Telemedicine.csv')
orals = pd.read_csv('Enhanced_MetPC_Orals.csv')
biomarkers = pd.read_csv('Enhanced_MetPC_Biomarkers.csv')
adt = pd.read_csv('Enhanced_MetPC_ADT.csv')
alpha_beta = pd.read_csv('Enhanced_MetPC_AlphaBetaEmitters.csv')
treat = pd.read_csv('Enhanced_MetPC_PrimaryTreatment.csv')
provenge = pd.read_csv('Enhanced_MetPC_Provenge.csv')

##### Visit and Telemedicine

In [58]:
visit_tele = (
    visit
    .drop(columns = ['VisitType', 'IsVitalsVisit', 'IsTreatmentVisit', 'IsLabVisit'])
    .append(telemedicine)
) 

In [59]:
visit_tele.loc[:,'VisitDate'] = pd.to_datetime(visit_tele['VisitDate'])

In [60]:
# Select max VisitDate from combined Visit and Telemedicine table.
visit_tele_max = (
    visit_tele
    [visit_tele['PatientID'].isin(test_IDs)]
    .groupby('PatientID')['VisitDate'].max()
    .to_frame(name = 'visit_max')
    .reset_index()
)

##### Orals

In [61]:
orals = orals[orals['PatientID'].isin(test_IDs)]

In [62]:
orals.loc[:, 'StartDate'] = pd.to_datetime(orals['StartDate'])

In [63]:
orals.loc[:, 'EndDate'] = pd.to_datetime(orals['EndDate'])

In [64]:
orals_max = (
    orals
    .assign(max_date = orals[['StartDate', 'EndDate']].max(axis = 1))
    .groupby('PatientID')['max_date'].max()
    .to_frame(name = 'orals_max')
    .reset_index()
)

##### Biomarkers

In [65]:
biomarkers = biomarkers[biomarkers['PatientID'].isin(test_IDs)]

In [66]:
biomarkers.loc[:, 'SpecimenCollectedDate'] = pd.to_datetime(biomarkers['SpecimenCollectedDate'])

In [67]:
biomarkers_max = (
    biomarkers
    .groupby('PatientID')['SpecimenCollectedDate'].max()
    .to_frame(name = 'biomarkers_max')
    .reset_index()
)

##### ADT

In [68]:
adt = adt[adt['PatientID'].isin(test_IDs)]

In [69]:
adt.loc[:, 'StartDate'] = pd.to_datetime(adt['StartDate'])

In [70]:
adt.loc[:, 'EndDate'] = pd.to_datetime(adt['EndDate'])

In [71]:
adt_max = (
    adt
    .assign(max_date = adt[['StartDate', 'EndDate']].max(axis = 1))
    .groupby('PatientID')['max_date'].max()
    .to_frame(name = 'adt_max')
    .reset_index()
)

##### Alpha Beta Emitters

In [72]:
alpha_beta = alpha_beta[alpha_beta['PatientID'].isin(test_IDs)]

In [73]:
alpha_beta.loc[:, 'AdministrationDate'] = pd.to_datetime(alpha_beta['AdministrationDate'])

In [74]:
alpha_beta_max = (
    alpha_beta
    .groupby('PatientID')['AdministrationDate'].max()
    .to_frame(name = 'alpha_beta_max')
    .reset_index()
)

##### Primary Treatment 

In [75]:
treat = treat[treat['PatientID'].isin(test_IDs)]

In [76]:
treat.loc[:, 'TreatmentDate'] = pd.to_datetime(treat['TreatmentDate'])

In [77]:
treat_max = (
    treat
    .groupby('PatientID')['TreatmentDate'].max()
    .to_frame(name = 'treat_max')
    .reset_index()
)

##### Provenge

In [78]:
provenge = provenge[provenge['PatientID'].isin(test_IDs)]

In [79]:
provenge.loc[:, 'StartDate'] = pd.to_datetime(provenge['StartDate'])

In [80]:
provenge_max = (
    provenge
    .groupby('PatientID')['StartDate'].max()
    .to_frame(name = 'provenge_max')
    .reset_index()
)

##### CRPC Date

In [81]:
crpc_max = enhanced_met[enhanced_met['CRPCDate'].notna()][['PatientID', 'CRPCDate']]

##### Max date merge

In [82]:
last_activity = pd.merge(visit_tele_max, orals_max, on = 'PatientID', how = 'outer')

In [83]:
last_activity = pd.merge(last_activity, biomarkers_max, on = 'PatientID', how = 'outer')

In [84]:
last_activity = pd.merge(last_activity, adt_max, on = 'PatientID', how = 'outer')

In [85]:
last_activity = pd.merge(last_activity, alpha_beta_max, on = 'PatientID', how = 'outer')

In [86]:
last_activity = pd.merge(last_activity, treat_max, on = 'PatientID', how = 'outer')

In [87]:
last_activity = pd.merge(last_activity, provenge_max, on = 'PatientID', how = 'outer')

In [88]:
last_activity = pd.merge(last_activity, crpc_max, on = 'PatientID', how = 'outer')

In [89]:
row_ID(last_activity)

(3786, 3786)

In [90]:
# Find max of each row. 
last_activity = (
    last_activity
    .assign(last_activity = last_activity[['visit_max', 'orals_max', 'biomarkers_max', 'adt_max', 'alpha_beta_max', 'treat_max', 'provenge_max', 'CRPCDate']].max(axis = 1))
    .filter(items = ['PatientID', 'last_activity'])
)

In [91]:
len(last_activity) == len(test_IDs)

True

In [92]:
last_activity['last_activity'].isna().sum()

0

In [93]:
# Append missing training IDs.
mortality = (
    mortality
    .append(
        pd.Series(test_IDs)[~pd.Series(test_IDs).isin(mortality['PatientID'])].to_frame(name = 'PatientID'), 
        sort = False
    )
)

In [94]:
row_ID(mortality)

(3786, 3786)

In [95]:
mortality = pd.merge(mortality, enhanced_met[['PatientID', 'met_date']], on = 'PatientID')

In [96]:
mortality = pd.merge(mortality, last_activity, on = 'PatientID')

In [97]:
row_ID(mortality)

(3786, 3786)

In [98]:
mortality.loc[:, 'death_status'] = np.where(mortality['death_date'].isna(), 0, 1)

In [99]:
# timerisk_activity is time from metastatic diagnosis to death or last activity if no death date.
mortality.loc[:, 'timerisk_activity'] = (
    np.where(mortality['death_date'].isna(),
             (mortality['last_activity'] - mortality['met_date']).dt.days,
             (mortality['death_date'] - mortality['met_date']).dt.days)
)

In [100]:
# If timerisk_activity is less than 0, set to 0 otherwise remains unchanged. 
mortality['timerisk_activity'] = np.where(mortality['timerisk_activity'] < 0, 0, mortality['timerisk_activity'])

In [101]:
mortality.sample(5)

,PatientID,death_date,met_date,last_activity,death_status,timerisk_activity
2744,FEEFEC3CA4C0A,NaT,2013-10-16,2022-09-19,0,3260.0
3519,F5EF462EB27F7,NaT,2018-10-03,2020-02-24,0,509.0
1928,FCEA38F474AD1,2021-08-23,2015-09-10,2021-05-05,1,2174.0
2862,F983A6339952B,NaT,2020-12-28,2022-09-30,0,641.0
931,F9FDED948FB67,2017-10-09,2016-06-27,2017-10-09,1,469.0


In [102]:
mortality = pd.merge(mortality, enhanced_met[['PatientID', 'diagnosis_date']], on = 'PatientID', how = 'outer')

In [103]:
# timerisk_activity_first is time from first diagnosis (metastatic or not) to death or last activity if no death date.
mortality.loc[:, 'timerisk_activity_first'] = (
    np.where(mortality['death_date'].isna(),
             (mortality['last_activity'] - mortality['diagnosis_date']).dt.days,
             (mortality['death_date'] - mortality['diagnosis_date']).dt.days)
)

In [104]:
# If timerisk_activity is less than 0, set to 0 otherwise remains unchanged. 
mortality['timerisk_activity_first'] = np.where(
    mortality['timerisk_activity_first'] < 0, 0, mortality['timerisk_activity_first'])

In [105]:
mortality.to_csv('mortality_cleaned_te.csv', index = False, header = True)

In [106]:
mortality = mortality.filter(items = ['PatientID', 'death_status', 'timerisk_activity'])

In [107]:
mortality.sample(5)

,PatientID,death_status,timerisk_activity
2949,F0CDF527312BA,0,1089.0
2410,F6830B456D25C,0,106.0
1130,F2AF6EF044748,1,1034.0
1375,F68A171A9A44E,1,1762.0
834,FC796CA5596E1,1,777.0


In [108]:
%whos DataFrame

Variable         Type         Data/Info
---------------------------------------
adt              DataFrame               PatientID Trea<...>\n[4080 rows x 5 columns]
adt_max          DataFrame              PatientID    ad<...>\n[3659 rows x 2 columns]
alpha_beta       DataFrame              PatientID    Dr<...>\n[1221 rows x 3 columns]
alpha_beta_max   DataFrame             PatientID alpha_<...>n\n[295 rows x 2 columns]
biomarkers       DataFrame              PatientID Bioma<...>\n[1498 rows x 9 columns]
biomarkers_max   DataFrame              PatientID bioma<...>\n[1016 rows x 2 columns]
crpc_max         DataFrame               PatientID   CR<...>\n[2303 rows x 2 columns]
demographics     DataFrame              PatientID Gende<...>\n[3786 rows x 6 columns]
enhanced_met     DataFrame               PatientID diag<...>n[3786 rows x 17 columns]
last_activity    DataFrame              PatientID last_<...>\n[3786 rows x 2 columns]
mortality        DataFrame              PatientID  deat<...>

In [109]:
# Keep demographics, enhanced_met, and mortality
del adt
del adt_max
del alpha_beta
del alpha_beta_max
del biomarkers
del biomarkers_max
del crpc_max
del last_activity
del orals
del orals_max
del provenge
del provenge_max
del telemedicine
del treat
del treat_max
del visit
del visit_tele
del visit_tele_max

### 4. MedicationAdministration

In [110]:
med_admin = pd.read_csv('MedicationAdministration.csv')

In [111]:
med_admin = med_admin[med_admin['PatientID'].isin(test_IDs)]

**An indicator variable will be created for key medications (ie., steroids, opioids, other pain meds, antibiotics, anticoagulation, diabetic medicaitons, etc.) around time of metastatic diagnosis. The elgibility window is -90 days from metastatic diagnosis +30.** 

In [112]:
med_admin = pd.merge(med_admin, enhanced_met[['PatientID', 'met_date']], on = 'PatientID', how = 'left')

In [113]:
med_admin.loc[:, 'AdministeredDate'] = pd.to_datetime(med_admin['AdministeredDate'])

In [114]:
# Select window of -90 and +30 days from metastatic diagnosis. 
med_admin_win = (
    med_admin
    .assign(med_admin_diff = (med_admin['AdministeredDate'] - med_admin['met_date']).dt.days)
    .query('med_admin_diff >= -90 and med_admin_diff <= 30')
)

#### Antineoplastic 

**No indicator variable created.** 

#### Antiemetic

**No indicator variable created.** 

#### Solution-fluid

**No indicator variable created.** 

#### Steroid

**Clinically relevant so will create glucocorticoid indicator variable.** 

In [115]:
med_admin_win.loc[:, 'steroid_diag'] = (
    np.where((med_admin_win['DrugCategory'] == 'steroid') & 
             ((med_admin_win['Route'] == 'Intravenous') | 
              (med_admin_win['Route'] == 'Oral') | 
              (med_admin_win['Route'] == 'Intrajejunal') |
              (med_admin_win['Route'] == 'Nasogastric') |
              (med_admin_win['Route'] == 'enteral')), 1, 0)
)

#### Pain

**Three indicator variables from 'pain agent' category will be created:**
- **opioid PO: patient recieved oral opioid (fentanyl patch included)**
- **non-opioid PO: patient received non-opioid orally**
- **pain IV: patient received pain medciation intravenously**

##### Opioid PO

In [116]:
# List of avialable opioids in the US. 
opioid_list = [
    'buprenorphine',
    'codeine',
    'fentanyl',
    'hydrocodone',
    'hydromorphone',
    'methadone',
    'morphine',
    'oxycodone',
    'oxymorphone',
    'tapentadol',
    'tramadol'
]

In [117]:
med_admin_win.loc[:, 'opioid_PO_diag'] = (
    np.where(((med_admin_win['Route'] == 'Oral') | 
              (med_admin_win['Route'] == 'Transdermal') | 
              (med_admin_win['Route'] == 'Sublingual')) &
             (med_admin_win['CommonDrugName'].str.contains('|'.join(opioid_list))), 1, 0)
)

##### Nonopioid PO

In [118]:
med_admin_win.loc[:, 'nonopioid_PO_diag'] = (
    np.where((med_admin_win['DrugCategory'] == 'pain agent') & 
             (med_admin_win['Route'] == 'Oral') & 
             (~med_admin_win['CommonDrugName'].str.contains('|'.join(opioid_list))), 1, 0)
)

##### Pain IV

In [119]:
med_admin_win.loc[:, 'pain_IV_diag'] = (
    np.where((med_admin_win['DrugCategory'] == 'pain agent') & 
             (med_admin_win['Route'] == 'Intravenous'), 1, 0)
)

#### Hematologic agent

**A therapeutic anticoagulation indicator variable will be created from the hematological agent category and will consist of:** 
* **Heparin: patients on a heparin gtt or other pareneral agents (eg., bivilrudin or argatroban)**
* **Enoxaparin: patients administered greater than prophylactic dosing of enoxaparin or other subcutaneous agents (eg., dalteparin or fondaparinux)**
* **DOAC: patients administered any dose of apxiaban, rivaroxabn, edoxaban, or dabigatran**
* **Warfarin: patients administered any dose of warfarin.**  

##### Heparin and other parenteral agents

In [120]:
med_admin_win.loc[:, 'heparin_diag'] = (
    np.where(((med_admin_win['CommonDrugName'].str.contains('heparin')) & 
              (med_admin_win['AdministeredUnits'] == 'unit/kg/hr')) | 
             (med_admin_win['CommonDrugName'].str.contains('bivalirudin')) | 
             (med_admin_win['CommonDrugName'].str.contains('argatroban')), 1, 0)
)

###### Enoxaparin and other subcutaneous agents 

In [121]:
med_admin_win.loc[:, 'enoxaparin_diag'] = (
    np.where(((med_admin_win['CommonDrugName'].str.contains('enoxaparin')) & 
              (med_admin_win['AdministeredAmount'] > 40)) | 
             ((med_admin_win['CommonDrugName'].str.contains('dalteparin')) & 
              (med_admin_win['AdministeredAmount'] > 5000)) | 
             ((med_admin_win['CommonDrugName'].str.contains('fondaparinux')) & 
              (med_admin_win['AdministeredAmount'] > 2.5)), 1, 0)
)

##### DOAC

In [122]:
med_admin_win.loc[:, 'doac_diag'] = (
    np.where((med_admin_win['CommonDrugName'].str.contains('apixaban')) | 
             (med_admin_win['CommonDrugName'].str.contains('rivaroxaban')) | 
             (med_admin_win['CommonDrugName'].str.contains('dabigatran')) | 
             (med_admin_win['CommonDrugName'].str.contains('edoxaban')), 1, 0)
)

##### Warfarin

In [123]:
med_admin_win.loc[:, 'warfarin_diag'] = np.where((med_admin_win['CommonDrugName'].str.contains('warfarin')), 1, 0)

##### Anticoagulation merge 

In [124]:
# Combine heparin, enoxparin, DOAC, and warfarin columns into a single anticoagulation indicator variable. 
med_admin_win['ac_diag'] = (
    med_admin_win['heparin_diag'] + med_admin_win['enoxaparin_diag'] + med_admin_win['doac_diag'] + med_admin_win['warfarin_diag']
)

In [125]:
# Drop heparin, enoxaparin, DOAC, and warfarin columns. 
med_admin_win = med_admin_win.drop(columns = ['heparin_diag', 'enoxaparin_diag', 'doac_diag', 'warfarin_diag'])

##### DAPT

**No indicator variable created.** 

##### GCSF

**No indicator variable created.** 

##### Epoetin

**No indicator variable created.** 

##### tPA

**No indicator variable created.** 

#### Anti-infective 

**Clinically relevant; two separate indicator variables from the anti-infective drug category will be created:** 
* **antiinfective IV group** 
* **antiinfective PO group**

##### Anti-infective IV

In [126]:
med_admin_win.loc[:, 'antiinfective_IV_diag'] = (
    np.where((med_admin_win['DrugCategory'] == 'anti-infective') & 
             (med_admin_win['Route'] == 'Intravenous'), 1, 0)
)

##### Anti-infective PO

In [127]:
med_admin_win.loc[:, 'antiinfective_diag'] = (
    np.where((med_admin_win['DrugCategory'] == 'anti-infective') & 
             (med_admin_win['Route'] == 'Oral'), 1, 0)
)

#### Anesthetic

**No indicator variable created.** 

#### Cytoprotective

**No indicator variable created.** 

#### Antihyperglycemic

In [128]:
med_admin_win.loc[:, 'antihyperglycemic_diag'] = np.where(med_admin_win['DrugCategory'] == 'antihyperglycemic', 1, 0)

#### Proton pump inhibitor

In [129]:
med_admin_win.loc[:, 'ppi_diag'] = np.where(med_admin_win['DrugCategory'] == 'proton pump inhibitor', 1, 0)

#### Antidepressant

In [130]:
med_admin_win.loc[:, 'antidepressant_diag'] = np.where(med_admin_win['DrugCategory'] == 'antidepressant', 1, 0)

#### Bone therapy agent

In [131]:
med_admin_win.loc[:, 'bta_diag'] = np.where(med_admin_win['DrugCategory'] == 'bone therapy agent (bta)', 1, 0)

#### Hormone

In [132]:
med_admin_win.loc[:, 'thyroid_diag'] = np.where(med_admin_win['CommonDrugName'] == 'levothyroxine', 1, 0)

#### Gout and hyperurecemia agent 

**No indicator variable created.** 

#### 4.16 Immunosuppressive 

**No indicator variable created.** 

#### Sedative agent

**No indicator variable created.** 

#### Endocrine

**No indicator variable created.** 

#### Antidote and reversal agent

**No indicator variable created.** 

#### Hyperglycemic

**No indicator variable created.** 

#### Antithyroid agent

**No indicator variable created.** 

#### Anticholinergic

**No indicator variable created.** 

#### Calciumimetic

**No indicator variable created.** 

#### Targeted therapy

**No indicator variable created.** 

#### Condensing

In [133]:
# Select columns with indicator variables and PatientID, then collapse rows by PatientID and sum columns. 
med_admin_wide = (
    med_admin_win
    [med_admin_win.columns[med_admin_win.columns.str.contains('diag|PatientID')]]
    .groupby('PatientID').sum()
)

In [134]:
# Replace numbers greater than 1 with 1; 0 remains unchanged. 
med_admin_wide = (
    med_admin_wide.mask(med_admin_wide > 1, 1)
    .reset_index()
)

In [135]:
row_ID(med_admin_wide)

(1189, 1189)

In [136]:
# Append missing training IDs.
med_admin_wide = (
    med_admin_wide.append(
        pd.Series(test_IDs)[~pd.Series(test_IDs).isin(med_admin_wide['PatientID'])].to_frame(name = 'PatientID'),
        sort = False
    )
    .fillna(0)
)

In [137]:
row_ID(med_admin_wide)

(3786, 3786)

In [138]:
med_admin_wide.shape

(3786, 13)

In [139]:
med_admin_wide.sample(5)

,PatientID,steroid_diag,opioid_PO_diag,nonopioid_PO_diag,pain_IV_diag,ac_diag,antiinfective_IV_diag,antiinfective_diag,antihyperglycemic_diag,ppi_diag,antidepressant_diag,bta_diag,thyroid_diag
502,F97B3D2807992,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1148,FE8F4364CC4BF,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1060,FF85ACFFEB807,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1047,FE01F28308E34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1298,F0304873974E6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [140]:
%whos DataFrame

Variable         Type         Data/Info
---------------------------------------
demographics     DataFrame              PatientID Gende<...>\n[3786 rows x 6 columns]
enhanced_met     DataFrame               PatientID diag<...>n[3786 rows x 17 columns]
med_admin        DataFrame                PatientID    <...>143770 rows x 11 columns]
med_admin_wide   DataFrame              PatientID  ster<...>n[3786 rows x 13 columns]
med_admin_win    DataFrame                PatientID    <...>n[7486 rows x 24 columns]
mortality        DataFrame              PatientID  deat<...>\n[3786 rows x 3 columns]


In [141]:
# Keep demographics, enhanced_met, med_admin_wide, and mortality
del med_admin
del med_admin_win

### 5. Enhanced_MetPC_Biomarkers

In [142]:
biomarkers = pd.read_csv('Enhanced_MetPC_Biomarkers.csv')

In [143]:
biomarkers = biomarkers[biomarkers['PatientID'].isin(test_IDs)]

In [144]:
biomarkers.shape

(1498, 9)

**The Biomarkers dataframe is in a long format. The goal is to build a single-row-per-patient dataframe with columns reflecting a patient's biomarker status within a predefined elgibility window. For this project, the elgibility window is defined as negative infinity to +30 days from time of diagnosis of metastatic disease (ie., index date).** 

**Regarding biomarker date information, result date is the date the biomarker result was first reported, and so represents the date on which the clinician would be expected to have information about the patient’s biomarker status to inform the course of treatment. Flatiron recommends using result date as the relevant biomarker test date and using specimen received date as the proxy when result date is not available. The gaps between collected date and either received or result date are substantially more variable.**

**We'll begin by imputing specimen received date when result date is missing. Then, we'll select all biomarkers that fall within the elbility window.**

In [145]:
biomarkers.loc[:, 'ResultDate'] = pd.to_datetime(biomarkers['ResultDate'])

In [146]:
biomarkers.loc[:, 'SpecimenReceivedDate'] = pd.to_datetime(biomarkers['SpecimenReceivedDate'])

In [147]:
# Replace missing result date with specimen received date. 
biomarkers.loc[:, 'result_date'] = (
    np.where(biomarkers['ResultDate'].isna(), biomarkers['SpecimenReceivedDate'], biomarkers['ResultDate'])
)

In [148]:
biomarkers = pd.merge(biomarkers, enhanced_met[['PatientID', 'met_date']], on = 'PatientID', how = 'left')

In [149]:
# Create new variable that captures difference in days between result date and metastatic diagnosis. 
biomarkers.loc[:, 'bio_date_diff'] = (biomarkers['result_date'] - biomarkers['met_date']).dt.days

In [150]:
# Select all patients with biomarkers < +30 from metastatic diagnosis. 
biomarker_win = biomarkers[biomarkers['bio_date_diff'] <= 30]

**BRCA will be defined as positive if ever-positive and negative if always-negative.**

#### Assigning patient-level biomarker status 

In [151]:
# Create indicator variable where where 2 if positive, 1 if negative, and 0 if unknown or missing. 
conditions = [
    (biomarker_win['BiomarkerStatus'] == 'BRCA1 mutation identified') |
    (biomarker_win['BiomarkerStatus'] == 'BRCA2 mutation identified') |
    (biomarker_win['BiomarkerStatus'] == 'Both BRCA1 and BRCA2 mutations identified') |
    (biomarker_win['BiomarkerStatus'] == 'BRCA mutation NOS'),
    (biomarker_win['BiomarkerStatus'] == 'No BRCA mutation')]

choices = [2, 1]
biomarker_win.loc[:, 'brca_status'] = np.select(conditions, choices, default = 0)

/Users/xavierorcutt/Dropbox/ist_project/myenv/lib/python3.7/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


In [152]:
biomarker_wide = (
    biomarker_win
    .sort_values(by = ['PatientID', 'brca_status'], ascending = False)
    .drop_duplicates(subset = ['PatientID'], keep = 'first')
    [['PatientID', 'brca_status']]
)

In [153]:
row_ID(biomarker_wide)

(155, 155)

In [154]:
biomarker_wide['brca_status'] = (
    biomarker_wide['brca_status'].replace({
        2: 'positive',
        1: 'negative',
        0: 'unknown',
        np.nan: 'unknown'})
)

In [155]:
biomarker_wide = (
    biomarker_wide
    .append(
        pd.Series(test_IDs)[~pd.Series(test_IDs).isin(biomarker_wide['PatientID'])].to_frame(name = 'PatientID'),
        sort = False)
    .fillna('unknown')
)

In [156]:
row_ID(biomarker_wide)

(3786, 3786)

In [157]:
biomarker_wide.sample(5)

,PatientID,brca_status
3293,FE1E9F622929D,unknown
2940,F4A4B50580098,unknown
1275,FD809A0D7CA76,unknown
3310,F90AF8E8D313A,unknown
3529,F3BD80D3BBA40,unknown


In [158]:
%whos DataFrame

Variable         Type         Data/Info
---------------------------------------
biomarker_wide   DataFrame              PatientID brca_<...>\n[3786 rows x 2 columns]
biomarker_win    DataFrame              PatientID Bioma<...>\n[192 rows x 13 columns]
biomarkers       DataFrame              PatientID Bioma<...>n[1498 rows x 12 columns]
demographics     DataFrame              PatientID Gende<...>\n[3786 rows x 6 columns]
enhanced_met     DataFrame               PatientID diag<...>n[3786 rows x 17 columns]
med_admin_wide   DataFrame              PatientID  ster<...>n[3786 rows x 13 columns]
mortality        DataFrame              PatientID  deat<...>\n[3786 rows x 3 columns]


In [159]:
# Keep biomarker_wide, demographics, enhanced_met, med_admin_wide, and mortality
del biomarker_win
del biomarkers

### 6. Insurance

In [160]:
insurance = pd.read_csv('Insurance.csv')

In [161]:
insurance = insurance[insurance['PatientID'].isin(test_IDs)]

In [162]:
row_ID(insurance)

(12749, 3679)

**The insurance table contains patient insurance/payer information. Patients may have multiple payer categories concurrently. Start date is populated roughly 90% of the time, while end date is populated about 25% of the time. This mutiple-row-per-patient table will be transformed into a single-row-per-patient table. Indicator variables for each payer category active at time of metastatic diagnosis will be made as columns. Insurance will be considered active if start date is less than 30 days from metastatic diagnosis regardless of end date.** 

In [163]:
insurance.loc[:, 'StartDate'] = pd.to_datetime(insurance['StartDate'])

In [164]:
insurance = pd.merge(insurance, enhanced_met[['PatientID', 'met_date']], on = 'PatientID', how = 'left')

In [165]:
# Remove years with start dates less than 1920 which is likely a coding error. 
insurance = insurance[(insurance['StartDate']).dt.year >= 1920]

In [166]:
insurance.loc[:, 'insurance_date_diff'] = (insurance['StartDate'] - insurance['met_date']).dt.days

In [167]:
insurance_win = insurance[insurance['insurance_date_diff'] <= 30]

In [168]:
row_ID(insurance)

(11195, 3453)

In [169]:
# Recode payer category 
conditions = [
    (insurance_win['IsMedicareAdv'] == 'Yes') | 
    (insurance_win['IsPartAOnly'] == 'Yes') | 
    (insurance_win['IsPartBOnly'] == 'Yes') |
    (insurance_win['IsPartAandPartB'] == 'Yes') |
    (insurance_win['IsPartDOnly'] == 'Yes'),
    (insurance_win['IsManagedGovtPlan'] == 'Yes'),
    (insurance_win['IsManagedMedicaid'] == 'Yes'),
    (insurance_win['IsMedicareMedicaid'] == 'Yes')]

choices = ['Medicare', 'Other Government Program', 'Medicaid', 'medicare_medicaid']

insurance_win.loc[:, 'payer_category'] = np.select(conditions, choices, insurance_win['PayerCategory'])

/Users/xavierorcutt/Dropbox/ist_project/myenv/lib/python3.7/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


#### Medicare

In [170]:
insurance_win.loc[:, 'medicare'] = np.where(insurance_win['payer_category'] == 'Medicare', 1, 0)

#### Medicaid

In [171]:
insurance_win.loc[:, 'medicaid'] = np.where(insurance_win['payer_category'] == 'Medicaid', 1, 0)

#### Medicare/Medicaid 

In [172]:
insurance_win.loc[:, 'medicare_medicaid'] = np.where(insurance_win['payer_category'] == 'medicare_medicaid', 1, 0)

#### Commercial 

In [173]:
insurance_win.loc[:, 'commercial'] = np.where(insurance_win['payer_category'] == 'Commercial Health Plan', 1, 0)

#### Patient Assistance Programs 

In [174]:
insurance_win.loc[:, 'patient_assistance'] = np.where(insurance_win['payer_category'] == 'Patient Assistance Program', 1, 0)

#### Other Government Program 

In [175]:
insurance_win.loc[:, 'other_govt'] = np.where(insurance_win['payer_category'] == 'Other Government Program', 1, 0)

#### Self Pay 

In [176]:
insurance_win.loc[:, 'self_pay'] = np.where(insurance_win['payer_category'] == 'Self Pay', 1, 0)

#### Other Payer

In [177]:
insurance_win.loc[:, 'other'] = np.where(insurance_win['payer_category'] == 'Other Payer - Type Unknown', 1, 0)

#### Condense 

In [178]:
# After dropping 'insurance_date_diff', add columns by PatientID.
insurance_wide = (
    insurance_win
    .drop(columns = ['insurance_date_diff'])
    .groupby('PatientID').sum()
)

In [179]:
# Set any value greater than 1 to 1; leave 0 unchanged. 
insurance_wide = (
    insurance_wide
    .mask(insurance_wide > 1, 1)
    .reset_index()
)

In [180]:
row_ID(insurance_wide)

(2792, 2792)

In [181]:
# Append missing training IDs.
insurance_wide = (
    insurance_wide
    .append(
        pd.Series(test_IDs)[~pd.Series(test_IDs).isin(insurance_wide['PatientID'])].to_frame(name = 'PatientID'),
        sort = False)
)

In [182]:
row_ID(insurance_wide)

(3786, 3786)

In [183]:
insurance_wide = insurance_wide.fillna(0)

In [184]:
insurance_wide.sample(5)

,PatientID,medicare,medicaid,medicare_medicaid,commercial,patient_assistance,other_govt,self_pay,other
1964,FB33F0DC5E7AB,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2222,FCB6EC5C05B22,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
579,F320850E3168C,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1502,F89D83090DBE2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2221,FCB6D6DFAC379,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [185]:
%whos DataFrame

Variable         Type         Data/Info
---------------------------------------
biomarker_wide   DataFrame              PatientID brca_<...>\n[3786 rows x 2 columns]
demographics     DataFrame              PatientID Gende<...>\n[3786 rows x 6 columns]
enhanced_met     DataFrame               PatientID diag<...>n[3786 rows x 17 columns]
insurance        DataFrame               PatientID     <...>[11195 rows x 15 columns]
insurance_wide   DataFrame              PatientID  medi<...>\n[3786 rows x 9 columns]
insurance_win    DataFrame               PatientID     <...>n[5724 rows x 24 columns]
med_admin_wide   DataFrame              PatientID  ster<...>n[3786 rows x 13 columns]
mortality        DataFrame              PatientID  deat<...>\n[3786 rows x 3 columns]


In [186]:
# Keep biomarker_wide, demographics, enhanced_met, insurance_wide, med_admin_wide, and mortality
del insurance
del insurance_win

### 7. ECOG

In [187]:
ecog = pd.read_csv('ECOG.csv')

In [188]:
ecog = ecog[ecog['PatientID'].isin(test_IDs)]

In [189]:
row_ID(ecog)

(51606, 2905)

In [190]:
ecog.sample(5)

,PatientID,EcogDate,EcogValue
216075,FA8E1C6B36463,2020-10-28,0
69517,F6309BAA73D3F,2017-12-22,1
172811,F2190205E52BA,2020-09-30,2
127623,FFA9C96FF4767,2018-06-05,1
97906,F18C44D5204AB,2017-08-25,0


**The ECOG table is a longitudinal record of structured ECOG scores captured in the EHR for each patient. Many patients have multiple ECOG scores reported. A new dataframe will be built where one ECOG score will be assigned to each patient. The index date will be date of advanced diagnosis with an elgible window period of +30 days to -90 days from advanced diagnosis. The ECOG score closest to index date will be assigned to the patient. In the case of two ECOG scores on the same day or equidistant but on opposite sides of the index date, the higher ECOG score (worse performance) will be selected.** 

**BaselineECOG is a composite table that selects one ECOG score within +7 days and -30 days of a line of therapy. Patients might have two baseline ECOG values for line number 1 due to maintenance therapy. BaselineECOG will not be used for creating baseline models.** 

In [191]:
ecog = pd.merge(ecog, enhanced_met[['PatientID', 'met_date']], on = 'PatientID', how = 'left')

In [192]:
ecog.loc[:, 'EcogDate'] = pd.to_datetime(ecog['EcogDate'])      

In [193]:
ecog.loc[:, 'ecog_date_diff'] = (ecog['EcogDate'] - ecog['met_date']).dt.days

In [194]:
ecog_win = ecog[(ecog['ecog_date_diff'] >= -90) & (ecog['ecog_date_diff'] <= 30)]

In [195]:
# Time from metastatic diagnosis to ECOG date will be converted to an absolute value. 
ecog_win.loc[:, 'ecog_date_diff'] = ecog_win['ecog_date_diff'].abs()

/Users/xavierorcutt/Dropbox/ist_project/myenv/lib/python3.7/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [196]:
# Sort values with ECOG nearest to time of diagnosis as top row (and largest ECOG if multiple ECOGs that day) then select top row.   ECOG date nearest to day of diagnosis as top row and largest ES
ecog_diagnosis_wide = (
    ecog_win
    .sort_values(by = ['PatientID', 'ecog_date_diff', 'EcogValue'], ascending = [True, True, False])
    .drop_duplicates(subset = ['PatientID'], keep = 'first' )
    .filter(items = ['PatientID', 'EcogValue'])
    .rename(columns = {'EcogValue': 'ecog_diagnosis'})
)

In [197]:
row_ID(ecog_diagnosis_wide)

(1183, 1183)

In [198]:
# Append missing training IDs. 
ecog_diagnosis_wide = (
    ecog_diagnosis_wide
    .append(
        pd.Series(test_IDs)[~pd.Series(test_IDs).isin(ecog_diagnosis_wide['PatientID'])].to_frame(name = 'PatientID'),
        sort = False)
    .fillna('unknown')
)

In [199]:
row_ID(ecog_diagnosis_wide)

(3786, 3786)

In [200]:
ecog_diagnosis_wide.sample(5)

,PatientID,ecog_diagnosis
2591,F1982EFAF14C2,unknown
3646,FB6574DFC583F,unknown
3111,F3243E34B83CD,unknown
2149,FF0877FF13AAB,unknown
3026,F22EF6ACCA41B,unknown


In [201]:
%whos DataFrame

Variable              Type         Data/Info
--------------------------------------------
biomarker_wide        DataFrame              PatientID brca_<...>\n[3786 rows x 2 columns]
demographics          DataFrame              PatientID Gende<...>\n[3786 rows x 6 columns]
ecog                  DataFrame               PatientID   Ec<...>n[51606 rows x 5 columns]
ecog_diagnosis_wide   DataFrame               PatientID ecog<...>\n[3786 rows x 2 columns]
ecog_win              DataFrame               PatientID   Ec<...>\n[2669 rows x 5 columns]
enhanced_met          DataFrame               PatientID diag<...>n[3786 rows x 17 columns]
insurance_wide        DataFrame              PatientID  medi<...>\n[3786 rows x 9 columns]
med_admin_wide        DataFrame              PatientID  ster<...>n[3786 rows x 13 columns]
mortality             DataFrame              PatientID  deat<...>\n[3786 rows x 3 columns]


In [202]:
# Keep biomarker_wide, demographics, ecog_diagnosis_wide, enhanced_met, insurance_wide, med_admin_wide, and mortality
del ecog
del ecog_win

### 8. Vitals

In [203]:
vitals = pd.read_csv('Vitals.csv')

In [204]:
vitals = vitals[vitals['PatientID'].isin(test_IDs)]

In [205]:
row_ID(vitals)

(734652, 3777)

**The Vitals table is a longitudinal record of vitals captured in the EHR for each patient. A weight and BMI variable at time of advanced diagnosis will be created. The elgibility window will be -90 days to +30 days from advanced diagnosis. Average height from all visits will be used to calculate BMI. In the case of two weights on the same day or equidistant but on opposite sides of the index date, the lowest weight will be selected. Percent change in weight and weight slope 3 months within metastatic diagnosis will be calculated. Patients must have at least two weight recordings to calculate percent change in weight or weight slope.** 

#### Weight and BMI

In [206]:
# Create weight dataframe; remove weight values that are empty or equal to zero.
weight = (
    vitals
    .query('Test == "body weight"')
    .filter(items = ['PatientID', 'TestDate', 'TestResultCleaned'])
    .rename(columns = {'TestResultCleaned': 'weight'})
    .dropna(subset = ['weight'])
    .query('weight != 0')
)

In [207]:
weight.loc[:, 'TestDate'] = pd.to_datetime(weight['TestDate'])

In [208]:
weight = pd.merge(weight, enhanced_met[['PatientID', 'met_date']], on = 'PatientID', how = 'left')

In [209]:
# Weight elgibliity window is -90 and +30 from metastatic diagnosis diagnosis. 
weight_win_bmi = (
    weight
    .assign(weight_date_diff = (weight['TestDate'] - weight['met_date']).dt.days)
    .query('weight_date_diff >= -90 and weight_date_diff <= 30')
)

In [210]:
weight_win_bmi.loc[:, 'weight_date_diff'] = weight_win_bmi['weight_date_diff'].abs()

In [211]:
# Select weight closest to date of metastatic diagnosis; lowest weight selected in the event of two weights on same day or equidistant. 
weight_bmi_wide = (
    weight_win_bmi
    .sort_values(by = ['PatientID', 'weight_date_diff', 'weight'], ascending = [True, True, True])
    .drop_duplicates(subset = ['PatientID'], keep = 'first')
    .filter(items = ['PatientID', 'weight'])
    .rename(columns = {'weight': 'weight_diag'})
)

In [212]:
# Dataframe of average height for each patient. 
height_avg = (
    vitals
    .query('Test == "body height"')
    .filter(items = ['PatientID', 'TestResultCleaned'])
    .query('TestResultCleaned > 0')
    .groupby('PatientID')['TestResultCleaned'].mean()
    .to_frame()
    .reset_index()
    .rename(columns = {'TestResultCleaned': 'height_avg'})
)

In [213]:
weight_bmi_wide = pd.merge(weight_bmi_wide, height_avg, on = 'PatientID', how = 'left')

In [214]:
# Create BMI column. 
weight_bmi_wide = (
    weight_bmi_wide
    .assign(bmi_diag = lambda x: (x['weight_diag']/(x['height_avg']*x['height_avg']))*10000)
    .drop(columns = ['height_avg'])
)

In [215]:
# Append excluded IDs from training set and create a missing variable for those without BMI at diagnosis. 
weight_bmi_wide = (
    weight_bmi_wide
    .append(
        pd.Series(test_IDs)[~pd.Series(test_IDs).isin(weight_bmi_wide['PatientID'])].to_frame(name = 'PatientID'),
        sort = False)
)

In [216]:
row_ID(weight_bmi_wide)

(3786, 3786)

In [217]:
weight_bmi_wide.loc[:, 'bmi_diag_na'] = np.where(weight_bmi_wide['bmi_diag'].isna(), 1, 0)

In [218]:
weight_bmi_wide.sample(5)

,PatientID,weight_diag,bmi_diag,bmi_diag_na
3348,FA78DE4C513CB,NaN,NaN,1
1479,FB411CB5B113A,76.657048,26.170558,0
579,F467348D8FFE5,56.971155,17.517450,0
3363,F7BCF0A731126,NaN,NaN,1
1562,F06CE0544475C,NaN,NaN,1


#### Percent change 

In [219]:
# Select elgbility window of -90 to +90 days from metastatic diagnosis.
weight_win_summary = (
    weight
    .assign(weight_date_diff = (weight['TestDate'] - weight['met_date']).dt.days)
    .query('weight_date_diff >= -90 and weight_date_diff <= 90')
)

In [220]:
# Select patients with more than 1 weight recording within elgibility window.
weight_win_summary = weight_win_summary[weight_win_summary.duplicated(subset = ['PatientID'], keep = False)]

In [221]:
# Select weight from the earliest time within elgibility window. 
weight_tmin = weight_win_summary.loc[weight_win_summary.groupby('PatientID')['weight_date_diff'].idxmin()]

In [222]:
# Select weight from the latest time within elgibility window. 
weight_tmax = weight_win_summary.loc[weight_win_summary.groupby('PatientID')['weight_date_diff'].idxmax()]

In [223]:
# Combine above two dataframes and sort from earliest recorded weight to latest recorded weight for each patient. 
weight_tcomb = (
    pd.concat([weight_tmin, weight_tmax])
    .sort_values(by = ['PatientID', 'weight_date_diff'], ascending = True)
)

In [224]:
row_ID(weight_tcomb)

(4602, 2301)

In [225]:
weight_tcomb.loc[:, 'weight_pct_change'] = weight_tcomb.groupby('PatientID')['weight'].pct_change()

In [226]:
weight_tcomb.loc[:, 'diff_date_diff'] = weight_tcomb['weight_date_diff'].diff()

In [227]:
# Drop empty rows for weight_pct_change.
weight_pct_wide = (
    weight_tcomb
    .dropna(subset = ['weight_pct_change'])
    .filter(items = ['PatientID', 'weight_pct_change', 'diff_date_diff'])
)

In [228]:
row_ID(weight_pct_wide)

(2301, 2301)

In [229]:
# Append missing training IDs and create a missing variable for those without weight_pct_change. 
weight_pct_wide = (
    weight_pct_wide
    .append(
        pd.Series(test_IDs)[~pd.Series(test_IDs).isin(weight_pct_wide['PatientID'])].to_frame(name = 'PatientID'),
        sort = False)
    .drop(columns = ['diff_date_diff'])
)

In [230]:
row_ID(weight_pct_wide)

(3786, 3786)

In [231]:
weight_pct_wide.loc[:, 'weight_pct_na'] = np.where(weight_pct_wide['weight_pct_change'].isna(), 1, 0)

In [232]:
weight_pct_wide.sample(5)

,PatientID,weight_pct_change,weight_pct_na
1997,FA91434830A66,NaN,1
1196,FC0F21660A789,0.004502,0
4149,F41B931F7B426,0.027165,0
17622,F06CE0544475C,0.000000,0
16862,FCC2C3E5EC54F,-0.004975,0


#### Weight slope

In [233]:
from scipy.stats import linregress 

In [234]:
weight_win_summary.loc[:, 'date_ordinal'] = weight_win_summary['TestDate'].map(dt.datetime.toordinal)

In [235]:
# Dataframe of slope for weight recordings within window period (kg/day).
weight_slope_wide = (
    weight_win_summary
    .groupby('PatientID')
    .apply(lambda x: pd.Series(linregress(x['date_ordinal'], x['weight'])))
    .rename(columns = {0: 'weight_slope'})
    .reset_index()
    .filter(items = ['PatientID', 'weight_slope']))   

/Users/xavierorcutt/Dropbox/ist_project/myenv/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:170: RuntimeWarning: invalid value encountered in double_scalars
  slope = ssxym / ssxm
/Users/xavierorcutt/Dropbox/ist_project/myenv/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:187: RuntimeWarning: divide by zero encountered in double_scalars
  slope_stderr = np.sqrt((1 - r**2) * ssym / ssxm / df)


In [236]:
row_ID(weight_slope_wide)

(2301, 2301)

In [237]:
# Append missing training IDs. 
weight_slope_wide = (
    weight_slope_wide
    .append(
        pd.Series(test_IDs)[~pd.Series(test_IDs).isin(weight_slope_wide['PatientID'])].to_frame(name = 'PatientID'),
        sort = False)
)

In [238]:
row_ID(weight_slope_wide)

(3786, 3786)

In [239]:
weight_slope_wide.sample(5)

,PatientID,weight_slope
44,F04E4F98E30A6,-0.048432
2223,FF763291C3B02,0.050694
1321,FFC3FE3A7A983,NaN
1070,F71D4B21CF41C,-0.016800
914,F60538EF8B1DC,0.009257


#### Weight merge 

In [240]:
weight_wide = pd.merge(weight_bmi_wide, weight_pct_wide, on = 'PatientID')

In [241]:
weight_wide = pd.merge(weight_wide, weight_slope_wide, on = 'PatientID')

In [242]:
row_ID(weight_wide)

(3786, 3786)

In [243]:
weight_wide.sample(5)

,PatientID,weight_diag,bmi_diag,bmi_diag_na,weight_pct_change,weight_pct_na,weight_slope
418,F3288F9647724,85.728888,30.505060,0,NaN,1,NaN
978,F74C1DAC2E072,99.790240,30.404023,0,-0.009009,0,-0.028630
1596,FC389E19A5820,53.433138,17.023734,0,-0.027165,0,-0.055827
2949,F57DAF087B7E6,NaN,NaN,1,NaN,1,NaN
2494,F19A57B53D51A,NaN,NaN,1,NaN,1,NaN


In [244]:
%whos DataFrame

Variable              Type         Data/Info
--------------------------------------------
biomarker_wide        DataFrame              PatientID brca_<...>\n[3786 rows x 2 columns]
demographics          DataFrame              PatientID Gende<...>\n[3786 rows x 6 columns]
ecog_diagnosis_wide   DataFrame               PatientID ecog<...>\n[3786 rows x 2 columns]
enhanced_met          DataFrame               PatientID diag<...>n[3786 rows x 17 columns]
height_avg            DataFrame              PatientID  heig<...>\n[3736 rows x 2 columns]
insurance_wide        DataFrame              PatientID  medi<...>\n[3786 rows x 9 columns]
med_admin_wide        DataFrame              PatientID  ster<...>n[3786 rows x 13 columns]
mortality             DataFrame              PatientID  deat<...>\n[3786 rows x 3 columns]
vitals                DataFrame                 PatientID   <...>734652 rows x 15 columns]
weight                DataFrame               PatientID   Te<...>n[97442 rows x 4 columns]


In [245]:
# Keep biomarker_wide, demographics, ecog_diagnosis_wide, enhanced_met, insurance_wide, med_admin_wide, mortality, 
# and weight_wide
del height_avg
del vitals
del weight
del weight_bmi_wide
del weight_pct_wide
del weight_slope_wide
del weight_tcomb
del weight_tmax
del weight_tmin
del weight_win_bmi
del weight_win_summary

### 9. Labs

In [246]:
lab = pd.read_csv('Lab.csv')

In [247]:
lab = lab[lab['PatientID'].isin(test_IDs)]

In [248]:
row_ID(lab)

(2302346, 3696)

**The Lab table is a longitudinal record of lab captured in the EHR with multiple-rows-per-patient. A single-patient-per-row table will be built focusing on the following NCCN recommended labs:** 
* **Creatinine -- (LOINC: 2160-0 and 38483-4)**
* **Hemoglobin -- (LOINC: 718-7 and 20509-6)**
* **White blood cell count -- (LOINC: 26464-8 and 6690-2)**
* **Neutrophil count -- (LOINC: 26499-4, 751-8, 30451-9, and 753-4)**
* **Albumin, serum -- (LOINC: 1751-7)**
* **Total bilirubin -- (LOINC: 42719-5 and 1975-2)**
* **Sodium — (LOINC: 2947-0 and 2951-2)**
* **Bicarb — (LOINC: 1963-8, 1959-6, 14627-4, 1960-4, and 2028-9)**
* **Calcium — (LOINC: 17861-6 and 49765-1)** 
* **AST — (LOINC: 1920-8)**
* **ALT — (LOINC: 1742-6, 1743-4, and 1744-2)**
* **Platelet -- (LOINC: 26515-7, 777-3, 778-1, and 49497-1)**
* **Potassium -- (LOINC: 6298-4 and 2823-3)**
* **Chloride -- (LOINC: 2075-0)**
* **BUN -- (LOINC: 3094-0)**
* **ALP -- (LOINC: 6768-6)**
* **PSA -- (LOINC: 2857-1)**

**The index date will be time of advanced diagnosis with an elgibility window of -90 days to +30 days. The lab value closest to the index date will be selected for each patient. The following summary statistics, using an elgibility window of negative infinity to +30 days from advanced diagnosis, will also be created for the above variables:** 
* **Max**
* **Min**
* **Mean**
* **Standard deviation** 
* **Slope**

#### 9.1 Baseline lab values

In [249]:
lab = pd.merge(lab, enhanced_met[['PatientID', 'met_date']], on = 'PatientID', how = 'left')

In [250]:
lab.loc[:, 'ResultDate'] = pd.to_datetime(lab['ResultDate']) 

In [251]:
# Select rows with clinically relevant labs.
lab_core = (
    lab[
    (lab['LOINC'] == "2160-0") |
    (lab['LOINC'] == "38483-4") | 
    (lab['LOINC'] == "718-7") |
    (lab['LOINC'] == "20509-6") |
    (lab['LOINC'] == "26464-8") |
    (lab['LOINC'] == "6690-2") |
    (lab['LOINC'] == "26499-4") |
    (lab['LOINC'] == "751-8") |
    (lab['LOINC'] == "30451-9") |
    (lab['LOINC'] == "753-4") |
    (lab['LOINC'] == "1751-7") |
    (lab['LOINC'] == "42719-5") |
    (lab['LOINC'] == "1975-2") |
    (lab['LOINC'] == "2947-0") |
    (lab['LOINC'] == "2951-2") |
    (lab['LOINC'] == "1963-8") |
    (lab['LOINC'] == "1959-6") |
    (lab['LOINC'] == "14627-4") |
    (lab['LOINC'] == "1960-4") |
    (lab['LOINC'] == "2028-9") |
    (lab['LOINC'] == "17861-6") |
    (lab['LOINC'] == "49765-1") |
    (lab['LOINC'] == "1920-8") |
    (lab['LOINC'] == "1742-6") | 
    (lab['LOINC'] == "1743-4") |
    (lab['LOINC'] == "1744-2") |
    (lab['LOINC'] == "26515-7") | 
    (lab['LOINC'] == "777-3") |
    (lab['LOINC'] == "778-1") |
    (lab['LOINC'] == "49497-1") | 
    (lab['LOINC'] == "6298-4") |
    (lab['LOINC'] == "2823-3") |
    (lab['LOINC'] == "2075-0") | 
    (lab['LOINC'] == "3094-0") | 
    (lab['LOINC'] == "6768-6") | 
    (lab['LOINC'] == "2857-1")]
    .filter(items = ['PatientID', 
                     'ResultDate', 
                     'LOINC', 
                     'LabComponent', 
                     'TestUnits', 
                     'TestUnitsCleaned', 
                     'TestResult', 
                     'TestResultCleaned', 
                     'met_date'])
)

In [252]:
conditions = [
    ((lab_core['LOINC'] == '2160-0') | (lab_core['LOINC'] == '38483-4')),
    ((lab_core['LOINC'] == '718-7') | (lab_core['LOINC'] == '20509-6')),
    ((lab_core['LOINC'] == '26464-8') | (lab_core['LOINC'] == '6690-2')), 
    ((lab_core['LOINC'] == '26499-4') | (lab_core['LOINC'] == '751-8') | (lab_core['LOINC'] == '30451-9') | (lab_core['LOINC'] == '753-4')),
    (lab_core['LOINC'] == '1751-7'),
    ((lab_core['LOINC'] == '42719-5') | (lab_core['LOINC'] == '1975-2')),
    ((lab_core['LOINC'] == '2947-0') | (lab_core['LOINC'] == '2951-2')),
    ((lab_core['LOINC'] == '1963-8') | (lab_core['LOINC'] == '1959-6') | (lab_core['LOINC'] == '14627-4') | (lab_core['LOINC'] == '1960-4') | (lab_core['LOINC'] == '2028-9')),
    ((lab_core['LOINC'] == '17861-6') | (lab_core['LOINC'] == '49765-1')),
    (lab_core['LOINC'] == '1920-8'),
    ((lab_core['LOINC'] == '1742-6') | (lab_core['LOINC'] == '1743-4') | (lab_core['LOINC'] == '1744-2')),
    ((lab_core['LOINC'] == '26515-7') | (lab_core['LOINC'] == '777-3') | (lab_core['LOINC'] == '778-1') | (lab_core['LOINC'] == '49497-1')),
    ((lab_core['LOINC'] == '6298-4') | (lab_core['LOINC'] == '2823-3')),
    (lab_core['LOINC'] == '2075-0'), 
    (lab_core['LOINC'] == '3094-0'),
    (lab_core['LOINC'] == '6768-6'),
    (lab_core['LOINC'] == '2857-1')]

choices = ['creatinine', 
           'hemoglobin', 
           'wbc', 
           'neutrophil_count',  
           'albumin', 
           'total_bilirubin', 
           'sodium', 
           'bicarb',
           'calcium',
           'ast', 
           'alt',
           'platelet',
           'potassium', 
           'chloride',
           'bun',
           'alp',
           'psa']

lab_core.loc[:, 'lab_name'] = np.select(conditions, choices)

In [253]:
# Remove missing lab values. 
lab_core = lab_core.dropna(subset = ['TestResultCleaned'])

In [254]:
conditions = [
    ((lab_core['lab_name'] == 'wbc') | (lab_core['lab_name'] == 'neutrophil_count') | (lab_core['lab_name'] == 'platelet')) & 
    (lab_core['TestUnits'] == '10*3/L'),
    (lab_core['lab_name'] == 'hemoglobin') & (lab_core['TestUnits'] == 'g/uL')]

choices = [lab_core['TestResultCleaned'] * 1000000,
           lab_core['TestResultCleaned'] / 100000]

lab_core.loc[:, 'test_result_cleaned'] = np.select(conditions, choices, default = lab_core['TestResultCleaned'])

In [255]:
# Elgibliity window is -90 and +30 from advanced diagnosis. 
lab_core_win = (
    lab_core
    .assign(lab_date_diff = (lab_core['ResultDate'] - lab_core['met_date']).dt.days)
    .query('lab_date_diff >= -90 and lab_date_diff <= 30')
    .filter(items = ['PatientID', 'ResultDate', 'TestResultCleaned', 'lab_name', 'met_date', 'test_result_cleaned', 'lab_date_diff'])
)

In [256]:
lab_core_win.loc[:, 'lab_date_diff'] = lab_core_win['lab_date_diff'].abs()

In [257]:
# Select lab closest to date of advanced diagnosis and pivot to a wide table. 
lab_diag_wide = (
    lab_core_win
    .loc[lab_core_win.groupby(['PatientID', 'lab_name'])['lab_date_diff'].idxmin()]
    .pivot(index = 'PatientID', columns = 'lab_name', values = 'test_result_cleaned')
    .reset_index()
    .rename(columns = {
        'albumin': 'albumin_diag',
        'creatinine': 'creatinine_diag',
        'hemoglobin': 'hemoglobin_diag',
        'neutrophil_count': 'neutrophil_count_diag',
        'total_bilirubin': 'total_bilirubin_diag',
        'wbc': 'wbc_diag',
        'sodium': 'sodium_diag', 
        'bicarb': 'bicarb_diag',
        'calcium': 'calcium_diag',
        'ast': 'ast_diag', 
        'alt': 'alt_diag',
        'platelet': 'platelet_diag',
        'potassium': 'potassium_diag',
        'chloride': 'chloride_diag',
        'bun': 'bun_diag',
        'alp': 'alp_diag'})
)

lab_diag_wide.columns.name = None

In [258]:
lab_diag_wide = lab_diag_wide.drop(columns = ['psa'])

In [259]:
row_ID(lab_diag_wide)

(1841, 1841)

In [260]:
lab_diag_wide = (
    lab_diag_wide
    .append(
        pd.Series(test_IDs)[~pd.Series(test_IDs).isin(lab_diag_wide['PatientID'])].to_frame(name = 'PatientID'),
        sort = False)
)

In [261]:
row_ID(lab_diag_wide)

(3786, 3786)

In [262]:
# Create missing variables for labs at time of diagnosis. 
for x in range (1, len(lab_diag_wide.columns)):
    lab_diag_wide.loc[:, lab_diag_wide.columns[x]+'_na'] = np.where(lab_diag_wide[lab_diag_wide.columns[x]].isna(), 1, 0)

In [263]:
list(lab_diag_wide.columns)

['PatientID',
 'albumin_diag',
 'alp_diag',
 'alt_diag',
 'ast_diag',
 'bicarb_diag',
 'bun_diag',
 'calcium_diag',
 'chloride_diag',
 'creatinine_diag',
 'hemoglobin_diag',
 'neutrophil_count_diag',
 'platelet_diag',
 'potassium_diag',
 'sodium_diag',
 'total_bilirubin_diag',
 'wbc_diag',
 'albumin_diag_na',
 'alp_diag_na',
 'alt_diag_na',
 'ast_diag_na',
 'bicarb_diag_na',
 'bun_diag_na',
 'calcium_diag_na',
 'chloride_diag_na',
 'creatinine_diag_na',
 'hemoglobin_diag_na',
 'neutrophil_count_diag_na',
 'platelet_diag_na',
 'potassium_diag_na',
 'sodium_diag_na',
 'total_bilirubin_diag_na',
 'wbc_diag_na']

#### Mean, max, min, and standard deviation

In [264]:
# Elgibility window is negative infinity to +30 from advanced diagnosis. 
lab_core_win_summ = (
    lab_core
    .assign(lab_date_diff = (lab_core['ResultDate'] - lab_core['met_date']).dt.days)
    .query('lab_date_diff <= 30')
    .filter(items = ['PatientID', 'ResultDate', 'TestResultCleaned', 'lab_name', 'met_date', 'test_result_cleaned', 'lab_date_diff'])
)

In [265]:
# Pivot table of average values for core labs during elgibility period of -90 to -30 days from advanced diagnosis. 
lab_avg_wide = (
    lab_core_win_summ
    .groupby(['PatientID', 'lab_name'])['test_result_cleaned'].mean()
    .to_frame()
    .reset_index()
    .pivot(index = 'PatientID', columns = 'lab_name', values = 'test_result_cleaned')
    .reset_index()
    .rename(columns = {
        'albumin': 'albumin_avg',
        'creatinine': 'creatinine_avg',
        'hemoglobin': 'hemoglobin_avg',
        'neutrophil_count': 'neutrophil_count_avg',
        'total_bilirubin': 'total_bilirubin_avg',
        'wbc': 'wbc_avg',
        'sodium': 'sodium_avg', 
        'bicarb': 'bicarb_avg',
        'calcium': 'calcium_avg',
        'ast': 'ast_avg', 
        'alt': 'alt_avg',
        'platelet': 'platelet_avg',
        'potassium': 'potassium_avg',
        'chloride': 'chloride_avg',
        'bun': 'bun_avg',
        'alp': 'alp_avg',
        'psa': 'psa_avg'})
)

lab_avg_wide.columns.name = None

In [266]:
row_ID(lab_avg_wide)

(2003, 2003)

In [267]:
# Pivot table of maximum values for core labs during elgibility period of -90 to -30 days from advanced diagnosis. 
lab_max_wide = (
    lab_core_win_summ
    .groupby(['PatientID', 'lab_name'])['test_result_cleaned'].max()
    .to_frame()
    .reset_index()
    .pivot(index = 'PatientID', columns = 'lab_name', values = 'test_result_cleaned')
    .reset_index()
    .rename(columns = {
        'albumin': 'albumin_max',
        'creatinine': 'creatinine_max',
        'hemoglobin': 'hemoglobin_max',
        'neutrophil_count': 'neutrophil_count_max',
        'total_bilirubin': 'total_bilirubin_max',
        'wbc': 'wbc_max', 
        'sodium': 'sodium_max', 
        'bicarb': 'bicarb_max',
        'calcium': 'calcium_max',
        'ast': 'ast_max', 
        'alt': 'alt_max',
        'platelet': 'platelet_max',
        'potassium': 'potassium_max',
        'chloride': 'chloride_max',
        'bun': 'bun_max', 
        'alp': 'alp_max', 
        'psa': 'psa_max'})
)

lab_max_wide.columns.name = None

In [268]:
row_ID(lab_max_wide)

(2003, 2003)

In [269]:
# Pivot table of minimum values for core labs during elgibility period of -90 to -30 days from advanced diagnosis. 
lab_min_wide = (
    lab_core_win_summ
    .groupby(['PatientID', 'lab_name'])['test_result_cleaned'].min()
    .to_frame()
    .reset_index()
    .pivot(index = 'PatientID', columns = 'lab_name', values = 'test_result_cleaned')
    .reset_index()
    .rename(columns = {
        'albumin': 'albumin_min',
        'creatinine': 'creatinine_min',
        'hemoglobin': 'hemoglobin_min',
        'neutrophil_count': 'neutrophil_count_min',
        'total_bilirubin': 'total_bilirubin_min',
        'wbc': 'wbc_min',
        'sodium': 'sodium_min', 
        'bicarb': 'bicarb_min',
        'calcium': 'calcium_min',
        'ast': 'ast_min', 
        'alt': 'alt_min',
        'platelet': 'platelet_min',
        'potassium': 'potassium_min',
        'chloride': 'chloride_min',
        'bun': 'bun_min',
        'alp': 'alp_min',
        'psa': 'psa_min'})
)

lab_min_wide.columns.name = None

In [270]:
row_ID(lab_min_wide)

(2003, 2003)

In [271]:
# Pivot table of standard deviation for core labs during elgibility period of -90 to -30 days from advanced diagnosis. 
lab_std_wide = (
    lab_core_win_summ
    .groupby(['PatientID', 'lab_name'])['test_result_cleaned'].std()
    .to_frame()
    .reset_index()
    .pivot(index = 'PatientID', columns = 'lab_name', values = 'test_result_cleaned')
    .reset_index()
    .rename(columns = {
        'albumin': 'albumin_std',
        'creatinine': 'creatinine_std',
        'hemoglobin': 'hemoglobin_std',
        'neutrophil_count': 'neutrophil_count_std',
        'total_bilirubin': 'total_bilirubin_std',
        'wbc': 'wbc_std',
        'sodium': 'sodium_std', 
        'bicarb': 'bicarb_std',
        'calcium': 'calcium_std',
        'ast': 'ast_std', 
        'alt': 'alt_std',
        'platelet': 'platelet_std',
        'potassium': 'potassium_std',
        'chloride': 'chloride_std',
        'bun': 'bun_std', 
        'alp': 'alp_std', 
        'psa': 'psa_std'})
)

lab_std_wide.columns.name = None

In [272]:
row_ID(lab_std_wide)

(2003, 2003)

In [273]:
lab_summary_wide = pd.merge(lab_avg_wide, lab_max_wide, on = 'PatientID', how = 'outer')

In [274]:
lab_summary_wide = pd.merge(lab_summary_wide, lab_min_wide, on = 'PatientID', how = 'outer')

In [275]:
lab_summary_wide = pd.merge(lab_summary_wide, lab_std_wide, on = 'PatientID', how = 'outer')

In [276]:
row_ID(lab_summary_wide)

(2003, 2003)

In [277]:
lab_summary_wide = (
    lab_summary_wide
    .append(
        pd.Series(test_IDs)[~pd.Series(test_IDs).isin(lab_summary_wide['PatientID'])].to_frame(name = 'PatientID'),
        sort = False)
)

In [278]:
row_ID(lab_summary_wide)

(3786, 3786)

In [279]:
lab_summary_wide.sample(5)

,PatientID,albumin_avg,alp_avg,alt_avg,ast_avg,bicarb_avg,bun_avg,calcium_avg,chloride_avg,creatinine_avg,...,chloride_std,creatinine_std,hemoglobin_std,neutrophil_count_std,platelet_std,potassium_std,psa_std,sodium_std,total_bilirubin_std,wbc_std
1804,FE55B61187A68,42.000000,178.000000,43.000000,24.000000,29.700000,NaN,9.800000,105.000000,NaN,...,NaN,NaN,0.989949,NaN,21.213203,NaN,NaN,NaN,NaN,NaN
2441,F3A2265DF2C24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1587,FC883EF163BAE,31.000000,312.000000,22.000000,61.000000,27.000000,24.000000,9.300000,99.000000,0.640000,...,NaN,NaN,NaN,0.325269,NaN,NaN,NaN,NaN,NaN,NaN
763,F5EB5BA72C88B,40.444444,90.666667,21.111111,19.555556,33.222222,12.555556,8.966667,96.777778,0.847778,...,2.728451,0.048677,0.504315,2.245885,102.179363,0.359784,NaN,2.538591,0.060093,2.560295
705,F56DBF6E24AB9,37.000000,255.500000,10.000000,16.000000,22.500000,17.500000,9.150000,104.500000,0.810000,...,0.707107,0.028284,0.282843,NaN,2.828427,0.353553,NaN,0.707107,0.070711,0.424264


In [280]:
list(lab_summary_wide.columns)

['PatientID',
 'albumin_avg',
 'alp_avg',
 'alt_avg',
 'ast_avg',
 'bicarb_avg',
 'bun_avg',
 'calcium_avg',
 'chloride_avg',
 'creatinine_avg',
 'hemoglobin_avg',
 'neutrophil_count_avg',
 'platelet_avg',
 'potassium_avg',
 'psa_avg',
 'sodium_avg',
 'total_bilirubin_avg',
 'wbc_avg',
 'albumin_max',
 'alp_max',
 'alt_max',
 'ast_max',
 'bicarb_max',
 'bun_max',
 'calcium_max',
 'chloride_max',
 'creatinine_max',
 'hemoglobin_max',
 'neutrophil_count_max',
 'platelet_max',
 'potassium_max',
 'psa_max',
 'sodium_max',
 'total_bilirubin_max',
 'wbc_max',
 'albumin_min',
 'alp_min',
 'alt_min',
 'ast_min',
 'bicarb_min',
 'bun_min',
 'calcium_min',
 'chloride_min',
 'creatinine_min',
 'hemoglobin_min',
 'neutrophil_count_min',
 'platelet_min',
 'potassium_min',
 'psa_min',
 'sodium_min',
 'total_bilirubin_min',
 'wbc_min',
 'albumin_std',
 'alp_std',
 'alt_std',
 'ast_std',
 'bicarb_std',
 'bun_std',
 'calcium_std',
 'chloride_std',
 'creatinine_std',
 'hemoglobin_std',
 'neutrophil_coun

#### Slope

In [281]:
lab_core_win_summ.loc[:, 'result_date_ordinal'] = lab_core_win_summ['ResultDate'].map(dt.datetime.toordinal)

In [282]:
lab_slope_wide = (
    lab_core_win_summ
    .groupby(['PatientID', 'lab_name'])
    .apply(lambda x: pd.Series(linregress(x['result_date_ordinal'], x['test_result_cleaned'])))
    .rename(columns = {0: 'slope'})
    .reset_index()
    .filter(items = ['PatientID', 'lab_name', 'slope'])
    .pivot(index = 'PatientID', columns = 'lab_name', values = 'slope')
    .reset_index()
    .rename(columns = {
        'albumin': 'albumin_slope',
        'creatinine': 'creatinine_slope',
        'hemoglobin': 'hemoglobin_slope',
        'neutrophil_count': 'neutrophil_count_slope',
        'total_bilirubin': 'total_bilirubin_slope',
        'wbc': 'wbc_slope',
        'sodium': 'sodium_slope', 
        'bicarb': 'bicarb_slope',
        'calcium': 'calcium_slope',
        'ast': 'ast_slope', 
        'alt': 'alt_slope',
        'platelet': 'platelet_slope',
        'potassium': 'potassium_slope',
        'chloride': 'chloride_slope',
        'bun': 'bun_slope',
        'alp': 'alp_slope',
        'psa': 'psa_slope'})
)

lab_slope_wide.columns.name = None

/Users/xavierorcutt/Dropbox/ist_project/myenv/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:170: RuntimeWarning: invalid value encountered in double_scalars
  slope = ssxym / ssxm
/Users/xavierorcutt/Dropbox/ist_project/myenv/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:184: RuntimeWarning: invalid value encountered in sqrt
  t = r * np.sqrt(df / ((1.0 - r + TINY)*(1.0 + r + TINY)))
/Users/xavierorcutt/Dropbox/ist_project/myenv/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:187: RuntimeWarning: invalid value encountered in double_scalars
  slope_stderr = np.sqrt((1 - r**2) * ssym / ssxm / df)
/Users/xavierorcutt/Dropbox/ist_project/myenv/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:187: RuntimeWarning: divide by zero encountered in double_scalars
  slope_stderr = np.sqrt((1 - r**2) * ssym / ssxm / df)


In [283]:
row_ID(lab_slope_wide)

(2003, 2003)

In [284]:
lab_slope_wide = (
    lab_slope_wide
    .append(
        pd.Series(test_IDs)[~pd.Series(test_IDs).isin(lab_slope_wide['PatientID'])].to_frame(name = 'PatientID'),
        sort = False)
)

In [285]:
# Create missing variables for lab slope. 
for x in range (1, len(lab_slope_wide.columns)):
    lab_slope_wide.loc[:, lab_slope_wide.columns[x]+'_na'] = np.where(lab_slope_wide[lab_slope_wide.columns[x]].isna(), 1, 0)

In [286]:
row_ID(lab_slope_wide)

(3786, 3786)

#### Merge

In [287]:
lab_wide = pd.merge(lab_diag_wide, lab_summary_wide, on = 'PatientID')

In [288]:
lab_wide = pd.merge(lab_wide, lab_slope_wide, on = 'PatientID')

In [289]:
row_ID(lab_wide)

(3786, 3786)

In [290]:
list(lab_wide.columns)

['PatientID',
 'albumin_diag',
 'alp_diag',
 'alt_diag',
 'ast_diag',
 'bicarb_diag',
 'bun_diag',
 'calcium_diag',
 'chloride_diag',
 'creatinine_diag',
 'hemoglobin_diag',
 'neutrophil_count_diag',
 'platelet_diag',
 'potassium_diag',
 'sodium_diag',
 'total_bilirubin_diag',
 'wbc_diag',
 'albumin_diag_na',
 'alp_diag_na',
 'alt_diag_na',
 'ast_diag_na',
 'bicarb_diag_na',
 'bun_diag_na',
 'calcium_diag_na',
 'chloride_diag_na',
 'creatinine_diag_na',
 'hemoglobin_diag_na',
 'neutrophil_count_diag_na',
 'platelet_diag_na',
 'potassium_diag_na',
 'sodium_diag_na',
 'total_bilirubin_diag_na',
 'wbc_diag_na',
 'albumin_avg',
 'alp_avg',
 'alt_avg',
 'ast_avg',
 'bicarb_avg',
 'bun_avg',
 'calcium_avg',
 'chloride_avg',
 'creatinine_avg',
 'hemoglobin_avg',
 'neutrophil_count_avg',
 'platelet_avg',
 'potassium_avg',
 'psa_avg',
 'sodium_avg',
 'total_bilirubin_avg',
 'wbc_avg',
 'albumin_max',
 'alp_max',
 'alt_max',
 'ast_max',
 'bicarb_max',
 'bun_max',
 'calcium_max',
 'chloride_max',

In [291]:
%whos DataFrame

Variable              Type         Data/Info
--------------------------------------------
biomarker_wide        DataFrame              PatientID brca_<...>\n[3786 rows x 2 columns]
demographics          DataFrame              PatientID Gende<...>\n[3786 rows x 6 columns]
ecog_diagnosis_wide   DataFrame               PatientID ecog<...>\n[3786 rows x 2 columns]
enhanced_met          DataFrame               PatientID diag<...>n[3786 rows x 17 columns]
insurance_wide        DataFrame              PatientID  medi<...>\n[3786 rows x 9 columns]
lab                   DataFrame                 PatientID   <...>302346 rows x 17 columns]
lab_avg_wide          DataFrame              PatientID  albu<...>n[2003 rows x 18 columns]
lab_core              DataFrame                 PatientID Re<...>074239 rows x 11 columns]
lab_core_win          DataFrame                 PatientID Re<...>n[52056 rows x 7 columns]
lab_core_win_summ     DataFrame                 PatientID Re<...>[164975 rows x 8 columns]


In [292]:
# Keep biomarker_wide, demographics, ecog_diagnosis_wide, enhanced_met, insurance_wide, lab_wide, med_admin_wide, 
# mortality, and weight_wide
del lab
del lab_avg_wide
del lab_core
del lab_core_win
del lab_core_win_summ
del lab_diag_wide
del lab_max_wide
del lab_min_wide
del lab_slope_wide
del lab_std_wide
del lab_summary_wide

### 10. Diagnosis

In [293]:
diagnosis = pd.read_csv('Diagnosis.csv')

In [294]:
diagnosis = diagnosis[diagnosis['PatientID'].isin(test_IDs)]

In [295]:
row_ID(diagnosis)

(124806, 3786)

In [296]:
diagnosis.sample(5)

,PatientID,DiagnosisDate,DiagnosisCode,DiagnosisCodeSystem,DiagnosisDescription
528203,F6464040326FC,2019-10-03,K29.60,ICD-10-CM,Other gastritis without bleeding
600998,F7ABC5231273B,2019-09-03,M17.9,ICD-10-CM,"Osteoarthritis of knee, unspecified"
107933,FE2C4E3EBA5EA,2015-04-28,V15.82,ICD-9-CM,Personal history of tobacco use
208883,FEBCBE23C8DCB,2016-10-20,R32,ICD-10-CM,Unspecified urinary incontinence
22928,F3BF9DD58BE9F,2013-10-11,I48.91,ICD-10-CM,Unspecified atrial fibrillation


**The Diagnosis table is in long format and has close to 14,000 unique ICD 9 and 10 codes. The median number of ICD codes per patient is 9 with a standard deviation of about 160 which shows the high varaibility in number of ICD codes per patent.** 

**ICD codes before metatstatic diagnosis and up to 30 days past diagnosis will be mapped to the Elixhauser comorbidity index. ("Coding Algorithms for Defining Comorbidities in ICD-9-CM and ICD-10 Administrative Data" by Quan et al is used as a guide for linking ICD codes to Elixhauser comorbidities.) Presence of concurrent or prior cancer diagnosis that is not lung cancer or metastasis and sites of metastases at time of diagnosis will also be created.** 

#### Elixhauser

In [297]:
diagnosis = pd.merge(diagnosis, enhanced_met[['PatientID', 'met_date']], on = 'PatientID', how = 'left')

In [298]:
diagnosis.loc[:, 'DiagnosisDate'] = pd.to_datetime(diagnosis['DiagnosisDate'])

In [299]:
diagnosis.loc[:, 'diagnosis_date_diff'] = (diagnosis['DiagnosisDate'] - diagnosis['met_date']).dt.days

In [300]:
# Remove decimal to make mapping to Elixhauser easier. 
diagnosis.loc[:, 'diagnosis_code'] = diagnosis['DiagnosisCode'].replace('\.', '', regex = True)

##### Elixhauser for ICD-9

In [301]:
# ICD-9 dataframe with unique codes for each patient. 
diagnosis_elix_9 = (
    diagnosis
    .query('diagnosis_date_diff <= 30')
    .query('DiagnosisCodeSystem == "ICD-9-CM"')
    .drop_duplicates(subset = (['PatientID', 'DiagnosisCode']), keep = 'first')
    .filter(items = ['PatientID', 'DiagnosisCode', 'diagnosis_code'])
)

In [302]:
row_ID(diagnosis_elix_9)

(7520, 1337)

In [303]:
diagnosis_elix_9.loc[:, 'chf'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('39891|'
                                                          '402(01|11|91)|'
                                                          '404(01|03|[19][13])|'
                                                          '42(5[456789]|8)'), 1, 0)
)

In [304]:
diagnosis_elix_9.loc[:, 'cardiac_arrhythmias'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('426([079]|1[023])|'
                                                          '427[012346789]|'
                                                          '7850|'
                                                          '996(01|04)|'
                                                          'V450|'
                                                          'V533'), 1, 0)
)

In [305]:
diagnosis_elix_9.loc[:, 'valvular_disease'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('0932|'
                                                          '39[4567]|'
                                                          '424|'
                                                          '746[3456]|'
                                                          'V422|'
                                                          'V433'), 1, 0)
)

In [306]:
diagnosis_elix_9.loc[:, 'pulmonary_circulation'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('41(5[01]|6|7[089])'), 1, 0)
)

In [307]:
diagnosis_elix_9.loc[:, 'peripheral_vascular'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('0930|'
                                                          '4373|'
                                                          '44([01]|3[123456789]|71)|'
                                                          '557[19]|'
                                                          'V434'), 1, 0)
)

In [308]:
diagnosis_elix_9.loc[:, 'htn_uncomplicated'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('401'), 1, 0)
)

In [309]:
diagnosis_elix_9.loc[:, 'htn_complicated'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('40[2345]'), 1, 0)
)

In [310]:
diagnosis_elix_9.loc[:, 'paralysis'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('3341|'
                                                          '34([23]|4[01234569])'), 1, 0)
)

In [311]:
diagnosis_elix_9.loc[:, 'other_neuro_disorders'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('33(19|2[01]|3([45]|92)|[45]|62)|'
                                                          '34([015]|8[13])|'
                                                          '78[04]3'), 1, 0)
)

In [312]:
diagnosis_elix_9.loc[:, 'chronic_pulmonary'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('416[89]|'
                                                          '49|'
                                                          '50([012345]|64|8[18])'), 1, 0)
)

In [313]:
diagnosis_elix_9.loc[:, 'diabetes_uncomplicated'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('250[0123]'), 1, 0)
)

In [314]:
diagnosis_elix_9.loc[:, 'diabetes_complicated'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('250[456789]'), 1, 0)
)

In [315]:
diagnosis_elix_9.loc[:, 'hypothyroidism'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('2409|'
                                                          '24([34]|6[18])'), 1, 0)
)

In [316]:
diagnosis_elix_9.loc[:, 'renal_failure'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('403[019]1|'
                                                          '404[019][23]|'
                                                          '58([56]|80)|'
                                                          'V4(20|51)|'
                                                          'V56'), 1, 0)
)

In [317]:
diagnosis_elix_9.loc[:, 'liver_disease'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('070(2[23]]|3[23]|44|54|6|9)|'
                                                          '456[012]|'
                                                          '57([01]|2[2345678]|3[3489])|'
                                                          'V427'), 1, 0)
)

In [318]:
diagnosis_elix_9.loc[:, 'peptic_ulcer_disease'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('53[1234][79]'), 1, 0)
)

In [319]:
diagnosis_elix_9.loc[:, 'aids_hiv'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('04[234]'), 1, 0)
)

In [320]:
diagnosis_elix_9.loc[:, 'lymphoma'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('20([012]|30)|'
                                                          '2386'), 1, 0)
)

In [321]:
diagnosis_elix_9.loc[:, 'metastatic_cancer'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('19[6789]'), 1, 0)
)

In [322]:
diagnosis_elix_9.loc[:, 'solid_tumor_wout_mets'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('1[456]|'
                                                          '17[012456789]|'
                                                          '18|'
                                                          '19([012345])'), 1, 0)
)

In [323]:
diagnosis_elix_9.loc[:, 'rheumatoid_arthritis'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('446|'
                                                          '7010|'
                                                          '71(0[0123489]|12|4|93)|'
                                                          '72([05]|85|889|930)'), 1, 0)
)

In [324]:
diagnosis_elix_9.loc[:, 'coagulopathy'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('28(6|7[1345])'), 1, 0)
)

In [325]:
diagnosis_elix_9.loc[:, 'obesity'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('2780'), 1, 0)
)

In [326]:
diagnosis_elix_9.loc[:, 'weight_loss'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('26[0123]|'
                                                          '7832|'
                                                          '7994'), 1, 0)
)

In [327]:
diagnosis_elix_9.loc[:, 'fluid_electrolyte'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('2(536|76)'), 1, 0)
)

In [328]:
diagnosis_elix_9.loc[:, 'blood_loss_anemia'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('2800'), 1, 0)
)

In [329]:
diagnosis_elix_9.loc[:, 'deficiency_anemia'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('28(0[123456789]|1)'), 1, 0)
)

In [330]:
diagnosis_elix_9.loc[:, 'alcohol_abuse'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('2652|'
                                                          '291[12356789]|'
                                                          '30(3[09]|50)|'
                                                          '3575|'
                                                          '4255|'
                                                          '5353|'
                                                          '571[0123]|'
                                                          '980|'
                                                          'V113'), 1, 0)
)

In [331]:
diagnosis_elix_9.loc[:, 'drug_abuse'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('292|'
                                                          '30(4|5[23456789])|'
                                                          'V6542'), 1, 0)
)

In [332]:
diagnosis_elix_9.loc[:, 'psychoses'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('2938|'
                                                          '296[0145]4|'
                                                          '29[578]'), 1, 0)
)

In [333]:
diagnosis_elix_9.loc[:, 'depression'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('296[235]|'
                                                          '3(004|09|11)'), 1, 0)
)

In [334]:
# Create variable that captures ICD-9 codes not included in Elixhauser. 
diagnosis_elix_9.loc[:, 'elixhauser_other'] = (
    np.where(diagnosis_elix_9.iloc[:, 3:].eq(0).all(1), 1, 0)
)

In [335]:
# Single-row-per-patient dataframe with columns as Elixhauser comorbidities. 
diagnosis_elix_9_wide = (
    diagnosis_elix_9
    .drop(columns = ['DiagnosisCode', 'diagnosis_code'])
    .groupby('PatientID').sum()
    .reset_index()
)

In [336]:
row_ID(diagnosis_elix_9_wide)

(1337, 1337)

##### Elixhauser for ICD-10

In [337]:
# ICD-10 dataframe with unique codes for each patient.  
diagnosis_elix_10 = (
    diagnosis
    .query('diagnosis_date_diff <= 30')
    .query('DiagnosisCodeSystem == "ICD-10-CM"')
    .drop_duplicates(subset = (['PatientID', 'DiagnosisCode']), keep = 'first')
    .filter(items = ['PatientID', 'DiagnosisCode', 'diagnosis_code'])
)

In [338]:
row_ID(diagnosis_elix_10)

(15741, 2067)

In [339]:
diagnosis_elix_10.loc[:, 'chf'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('I099|'
                                                           'I1(10|3[02])|'
                                                           'I255|'
                                                           'I4(2[056789]|3)|'
                                                           'I50|'
                                                           'P290'), 1, 0)
)

In [340]:
diagnosis_elix_10.loc[:, 'cardiac_arrhythmias'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('I4(4[123]|5[69]|[789])|'
                                                           'R00[018]|'
                                                           'T821|'
                                                           'Z[49]50'), 1, 0)
)

In [341]:
diagnosis_elix_10.loc[:, 'valvular_disease'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('A520|'
                                                           'I0([5678]|9[18])|'
                                                           'I3[456789]|'
                                                           'Q23[0123]|'
                                                           'Z95[234]'), 1, 0)
)

In [342]:
diagnosis_elix_10.loc[:, 'pulmonary_circulation'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('I2([67]|8[089])'), 1, 0)
)

In [343]:
diagnosis_elix_10.loc[:, 'peripheral_vascular'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('I7([01]|3[189]|71|9[02])|'
                                                           'K55[189]|'
                                                           'Z95[89]'), 1, 0)
)

In [344]:
diagnosis_elix_10.loc[:, 'htn_uncomplicated'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('I10'), 1, 0)
)

In [345]:
diagnosis_elix_10.loc[:, 'htn_complicated'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('I1[1235]'), 1, 0)
)

In [346]:
diagnosis_elix_10.loc[:, 'paralysis'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('G041|'
                                                           'G114|'
                                                           'G8(0[12]|[12]|3[012349])'), 1, 0)
)

In [347]:
diagnosis_elix_10.loc[:, 'other_neuro_disorders'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('G1[0123]|'
                                                           'G2([012]|5[45])|'
                                                           'G3(1[289]|[2567])|'
                                                           'G4[01]|'
                                                           'G93[14]|'
                                                           'R470|'
                                                           'R56'), 1, 0)
)

In [348]:
diagnosis_elix_10.loc[:, 'chronic_pulmonary'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('I27[89]|'
                                                           'J4[01234567]|'
                                                           'J6([01234567]|84)|'
                                                           'J70[13]'), 1, 0)
)

In [349]:
diagnosis_elix_10.loc[:, 'diabetes_uncomplicated'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('E1[01234][019]'), 1, 0)
)

In [350]:
diagnosis_elix_10.loc[:, 'diabetes_complicated'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('E1[01234][2345678]'), 1, 0)
)

In [351]:
diagnosis_elix_10.loc[:, 'hypothyroidism'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('E0[0123]|'
                                                           'E890'), 1, 0)
)

In [352]:
diagnosis_elix_10.loc[:, 'renal_failure'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('I1(20|31)|'
                                                           'N1[89]|'
                                                           'N250|'
                                                           'Z49[012]|'
                                                           'Z9(40|92)'), 1, 0)
)

In [353]:
diagnosis_elix_10.loc[:, 'liver_disease'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('B18|'
                                                           'I8(5|64)|'
                                                           'I982|'
                                                           'K7(0|1[13457]|[234]|6[023456789])|'
                                                           'Z944'), 1, 0)
)

In [354]:
diagnosis_elix_10.loc[:, 'peptic_ulcer_disease'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('K2[5678][79]'), 1, 0)
)

In [355]:
diagnosis_elix_10.loc[:, 'aids_hiv'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('B2[0124]'), 1, 0)
)

In [356]:
diagnosis_elix_10.loc[:, 'lymphoma'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('C8[123458]|'
                                                           'C9(0[02]|6)'), 1, 0)
)

In [357]:
diagnosis_elix_10.loc[:, 'metastatic_cancer'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('C(7[789]|80)'), 1, 0)
)

In [358]:
diagnosis_elix_10.loc[:, 'solid_tumor_wout_mets'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('C[01]|'
                                                           'C2[0123456]|'
                                                           'C3[01234789]|'
                                                           'C4[01356789]|'
                                                           'C5[012345678]|'
                                                           'C6|'
                                                           'C7[0123456]|'
                                                           'C97'), 1, 0)
)

In [359]:
diagnosis_elix_10.loc[:, 'rheumatoid_arthritis'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('L94[013]|'
                                                           'M0[568]|'
                                                           'M12[03]|'
                                                           'M3(0|1[0123]|[2345])|'
                                                           'M4(5|6[189])'), 1, 0)
)

In [360]:
diagnosis_elix_10.loc[:, 'coagulopathy'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('D6([5678]|9[13456])'), 1, 0)
)

In [361]:
diagnosis_elix_10.loc[:, 'obesity'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('E66'), 1, 0)
)

In [362]:
diagnosis_elix_10.loc[:, 'weight_loss'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('E4[0123456]|'
                                                           'R6(34|4)'), 1, 0)
)

In [363]:
diagnosis_elix_10.loc[:, 'fluid_electrolyte'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('E222|'
                                                           'E8[67]'), 1, 0)
)

In [364]:
diagnosis_elix_10.loc[:, 'blood_loss_anemia'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('D500'), 1, 0)
)

In [365]:
diagnosis_elix_10.loc[:, 'deficiency_anemia'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('D5(0[89]|[123])'), 1, 0)
)

In [366]:
diagnosis_elix_10.loc[:, 'alcohol_abuse'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('F10|'
                                                           'E52|'
                                                           'G621|'
                                                           'I426|'
                                                           'K292|'
                                                           'K70[039]|'
                                                           'T51|'
                                                           'Z502|'
                                                           'Z7(14|21)'), 1, 0)
)

In [367]:
diagnosis_elix_10.loc[:, 'drug_abuse'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('F1[12345689]|'
                                                           'Z7(15|22)'), 1, 0)
)

In [368]:
diagnosis_elix_10.loc[:, 'psychoses'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('F2[0234589]|'
                                                           'F3([01]2|15)'), 1, 0)
)

In [369]:
diagnosis_elix_10.loc[:, 'depression'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('F204|'
                                                           'F3(1[345]|[23]|41)|'
                                                           'F4[13]2'), 1, 0)
)

In [370]:
# Create variable that captures ICD-10 codes not included in Elixhauser. 
diagnosis_elix_10.loc[:, 'elixhauser_other'] = (
    np.where(diagnosis_elix_10.iloc[:, 3:].eq(0).all(1), 1, 0)
)

In [371]:
diagnosis_elix_10_wide = (
    diagnosis_elix_10
    .drop(columns = ['DiagnosisCode', 'diagnosis_code'])
    .groupby('PatientID').sum()
    .reset_index()
)

In [372]:
row_ID(diagnosis_elix_10_wide)

(2067, 2067)

In [373]:
# Merge Elixhauser 9 and 10 and sum by PatientID.
diagnosis_elixhauser = (
    pd.concat([diagnosis_elix_9_wide, diagnosis_elix_10_wide])
    .groupby('PatientID').sum()
)

In [374]:
# Create unqiue ICD count for each patient. 
diagnosis_elixhauser['icd_count'] = diagnosis_elixhauser.sum(axis = 1)  

In [375]:
# Other than unique ICD count, values greater than 1 are set to 1; 0 remains unchanged. 
diagnosis_elixhauser.iloc[:, :-1] = (
    diagnosis_elixhauser.iloc[:, :-1].mask(diagnosis_elixhauser.iloc[:, :-1] >1, 1)
)

In [376]:
diagnosis_elixhauser = diagnosis_elixhauser.reset_index()

In [377]:
row_ID(diagnosis_elixhauser)

(2797, 2797)

In [378]:
# Append missing training IDs.
diagnosis_elixhauser = (
    diagnosis_elixhauser
    .append(
        pd.Series(test_IDs)[~pd.Series(test_IDs).isin(diagnosis_elixhauser['PatientID'])].to_frame(name = 'PatientID'), 
        sort = False)
    .fillna(0)
)

In [379]:
row_ID(diagnosis_elixhauser)

(3786, 3786)

In [380]:
diagnosis_elixhauser.sample(5)

,PatientID,chf,cardiac_arrhythmias,valvular_disease,pulmonary_circulation,peripheral_vascular,htn_uncomplicated,htn_complicated,paralysis,other_neuro_disorders,...,weight_loss,fluid_electrolyte,blood_loss_anemia,deficiency_anemia,alcohol_abuse,drug_abuse,psychoses,depression,elixhauser_other,icd_count
1002,F3673CFC2EF54,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1853,FA8F982A3FC25,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,...,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,182.0
409,F2522A5749CDC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
1188,F690E05957713,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1887,F24C12D1EE935,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Other cancer 

##### ICD-9 Cancer codes 

In [381]:
# Select all ICD-9 cancer codes between 140-209.
# Exclude benign neoplasms: 210-229, carcinoma in site: 230-234, and neoplasms of uncertain behavior or nature: 235-239.
cancer_9 = (
    diagnosis_elix_9[diagnosis_elix_9['DiagnosisCode'].str.startswith(
        ('14','15', '16', '17', '18', '19', '20'))]
    .filter(items = ['PatientID', 'DiagnosisCode', 'diagnosis_code'])
)

In [382]:
row_ID(cancer_9)

(1655, 1142)

**Remove the following ICD-9 codes representing prostate cancer, metastasis, ill-defined neoplasms, and benign neoplasms of skin (BCC and SCC):**
* **173 - Other and unspecified malignant neoplasm of skin**
* **185 - Malignant neoplasm of the prostate**
* **196 - Secondary and unspecified malignant neoplasm of lymph nodes**
* **197 - Secondary malignant neoplasm of respiratory and digestive systems**
* **198 - Secondary malignant neoplasm of other specified sites** 
* **199 - Malignant neoplasm without specification of site**

In [383]:
# Dataframe of ICD-9 neoplasm codes that exclude colorectal cancer, metastasis, or benign neoplasms.
other_cancer_9 = (
    cancer_9[~cancer_9['diagnosis_code'].str.match('173|'
                                                   '185|'
                                                   '19([6789])')]
)

In [384]:
other_cancer_9.loc[:,'other_cancer_9'] = 1

/Users/xavierorcutt/Dropbox/ist_project/myenv/lib/python3.7/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


In [385]:
other_cancer_9 = (
    other_cancer_9
    .drop_duplicates(subset = 'PatientID', keep = 'first')
    .filter(items = ['PatientID', 'other_cancer_9'])
)

In [386]:
row_ID(other_cancer_9)

(154, 154)

In [387]:
other_cancer_9 = (
    other_cancer_9
    .append(
        pd.Series(test_IDs)[~pd.Series(test_IDs).isin(other_cancer_9['PatientID'])].to_frame(name = 'PatientID'), 
        sort = False)
    .fillna(0)
)

In [388]:
row_ID(other_cancer_9)

(3786, 3786)

##### ICD-10 Cancer codes

In [389]:
# Select all ICD-10 codes between C00-D49 
# Exclude in situ neoplasms: D00-D09, benign neoplasms: D10-D36, benign neuroendocrine tumor: D3A, and neoplasms of unspecified behavior: D37 and D49
cancer_10 = (
    diagnosis_elix_10[diagnosis_elix_10['DiagnosisCode'].str.startswith(
        ('C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'D38', 'D39', 'D4'))]
    .filter(items = ['PatientID', 'DiagnosisCode', 'diagnosis_code'])
)

In [390]:
row_ID(cancer_10)

(3224, 1851)

**Remove the following ICD-10 codes which capture breast cancer, metastasis, and benign skin neoplasms(eg., BCC and SCC).**
* **C61 - Malignant neoplasm of prostate** 
* **C44 - Other and unspecified malignant neoplasm of skin**
* **C77 - Secondary and unspecified malignant neoplasm of lymph nodes**
* **C78 - Secondary malignant neoplasm of respiratory and digestive organs**
* **C79 - Secondary malignant neoplasm of other and unspecified sites**
* **C80 - Malignant neoplasm without specification of site**
* **D47.2 - Monoclonal gammopathy**
* **D48 - Neoplasm of uncertain behavior of other and unspecified sites**
* **D49 - Neoplasms of unspecified behavior** 

In [391]:
# Dataframe of ICD-10 neoplasm codes that exclude prostate cancer, metastasis, or benign neoplasms.
other_cancer_10 = (
    cancer_10[~cancer_10['diagnosis_code'].str.match('C61|'
                                                     'C44|'
                                                     'C7[789]|'
                                                     'C80|'
                                                     'D4(72|[89])')]
)

In [392]:
other_cancer_10.loc[:,'other_cancer_10'] = 1

In [393]:
# Drop duplicates.
other_cancer_10 = (
    other_cancer_10
    .drop_duplicates(subset = 'PatientID', keep = 'first')
    .filter(items = ['PatientID', 'other_cancer_10'])
)

In [394]:
row_ID(other_cancer_10)

(176, 176)

In [395]:
# Append missing training IDs.
other_cancer_10 = (
    other_cancer_10
    .append(
        pd.Series(test_IDs)[~pd.Series(test_IDs).isin(other_cancer_10['PatientID'])].to_frame(name = 'PatientID'), 
        sort = False)
    .fillna(0)
)

In [396]:
row_ID(other_cancer_10)

(3786, 3786)

In [397]:
other_cancer = pd.merge(other_cancer_9, other_cancer_10, on = 'PatientID')

In [398]:
# Combine other_cancer_9 and other_cancer_19; replace values equal to 2 with 1. 
other_cancer = (
    other_cancer
    .assign(other_cancer = other_cancer['other_cancer_9'] + other_cancer['other_cancer_10'])
    .filter(items = ['PatientID', 'other_cancer'])
    .replace(2, 1)
)

In [399]:
row_ID(other_cancer)

(3786, 3786)

#### Sites of metastases

##### ICD-9 sites of metastases

In [400]:
# Create dataframe contianing patients with ICD-9 codes within -90 to +30 days from metastatic diagnosis and remove duplicate codes
diagnosis_mets_9 = (
    diagnosis
    .query('diagnosis_date_diff >= -90 and diagnosis_date_diff <= 30')
    .query('DiagnosisCodeSystem == "ICD-9-CM"')
    .drop_duplicates(subset = ['PatientID', 'DiagnosisCode'], keep = 'first')
    .filter(items = ['PatientID', 'DiagnosisCode', 'diagnosis_code'])
)

**Sites of metastasis will be grouped into the following categories according to ICD-9 codes:**
* **Thorax - 197.0, 197.1, 197.2, and 197.3**
* **Peritoneum - 197.6**
* **Liver - 197.7**
* **Other GI - 197.4 and 197.8**
* **CNS - 198.3 and 198.4**
* **Bone - 198.5**
* **Lymph -- 196**
* **Kidney, bladder -- 198.0 and 198.1**
* **Other - 198.2, 198.6, 198.7, and 198.8**

In [401]:
diagnosis_mets_9['thorax_met'] = np.where(diagnosis_mets_9['diagnosis_code'].str.match('197[0123]'), 1, 0)

In [402]:
diagnosis_mets_9['peritoneum_met'] = np.where(diagnosis_mets_9['diagnosis_code'].str.match('1976'), 1, 0)

In [403]:
diagnosis_mets_9['liver_met'] = np.where(diagnosis_mets_9['diagnosis_code'].str.match('1977'), 1, 0)

In [404]:
diagnosis_mets_9['other_gi_met'] = np.where(diagnosis_mets_9['diagnosis_code'].str.match('197[48]'), 1, 0)

In [405]:
diagnosis_mets_9['cns_met'] = np.where(diagnosis_mets_9['diagnosis_code'].str.match('198[34]'), 1, 0)

In [406]:
diagnosis_mets_9['bone_met'] = np.where(diagnosis_mets_9['diagnosis_code'].str.match('1985'), 1, 0)

In [407]:
diagnosis_mets_9['lymph_met'] = np.where(diagnosis_mets_9['diagnosis_code'].str.match('196'), 1, 0)

In [408]:
diagnosis_mets_9['kidney_bladder_met'] = np.where(diagnosis_mets_9['diagnosis_code'].str.match('198[01]'), 1, 0)

In [409]:
diagnosis_mets_9['other_met'] = np.where(diagnosis_mets_9['diagnosis_code'].str.match('198[2678]'), 1, 0)

In [410]:
# Collapse columns and sum.
diagnosis_mets_9 = (
    diagnosis_mets_9
    .drop(columns = ['DiagnosisCode', 'diagnosis_code'])
    .groupby('PatientID').sum()
    .reset_index()
)

##### ICD-10 sites of metastases

In [411]:
# Create dataframe contianing patients with ICD-10 codes within -90 to +30 days from advanced diagnosis and remove duplicate codes.
diagnosis_mets_10 = (
    diagnosis
    .query('diagnosis_date_diff >= -90 and diagnosis_date_diff <= 30')
    .query('DiagnosisCodeSystem == "ICD-10-CM"')
    .drop_duplicates(subset = ['PatientID', 'DiagnosisCode'], keep = 'first')
    .filter(items = ['PatientID', 'DiagnosisCode', 'diagnosis_code'])
)

**Sites of metastasis will be grouped into the following categories according to ICD-10 codes:**
* **Thorax - C78.0, C78.1, C78.2, and C78.3**
* **Peritoneum - C78.6**
* **Liver - C78.7**
* **Other GI - C78.4 and C78.8**
* **CNS - C79.3 and C79.4**
* **Bone - C79.5**
* **Lymph -- C77**
* **Kidney, bladder -- C79.0 and 79.1**
* **Other - C79.2, C79.6, C79.7, C79.8, and C79.9**

In [412]:
diagnosis_mets_10['thorax_met'] = np.where(diagnosis_mets_10['diagnosis_code'].str.match('C78[0123]'), 1, 0)

In [413]:
diagnosis_mets_10['peritoneum_met'] = np.where(diagnosis_mets_10['diagnosis_code'].str.match('C786'), 1, 0)

In [414]:
diagnosis_mets_10['liver_met'] = np.where(diagnosis_mets_10['diagnosis_code'].str.match('C787'), 1, 0)

In [415]:
diagnosis_mets_10['other_gi_met'] = np.where(diagnosis_mets_10['diagnosis_code'].str.match('C78[48]'), 1, 0)

In [416]:
diagnosis_mets_10['cns_met'] = np.where(diagnosis_mets_10['diagnosis_code'].str.match('C79[34]'), 1, 0)

In [417]:
diagnosis_mets_10['bone_met'] = np.where(diagnosis_mets_10['diagnosis_code'].str.match('C795'), 1, 0)

In [418]:
diagnosis_mets_10['lymph_met'] = np.where(diagnosis_mets_10['diagnosis_code'].str.match('C77'), 1, 0)

In [419]:
diagnosis_mets_10['kidney_bladder_met'] = np.where(diagnosis_mets_10['diagnosis_code'].str.match('C79[01]'), 1, 0)

In [420]:
diagnosis_mets_10['other_met'] = np.where(diagnosis_mets_10['diagnosis_code'].str.match('C79[26789]'), 1, 0)

In [421]:
# Collapse columns and sum. 
diagnosis_mets_10 = (
    diagnosis_mets_10
    .drop(columns = ['DiagnosisCode', 'diagnosis_code'])
    .groupby('PatientID').sum()
    .reset_index()
)

In [422]:
# Merge ICD-9 and ICD-10 mets tables; collapse and sum. 
mets_wide = (
    pd.concat([diagnosis_mets_9, diagnosis_mets_10])
    .groupby('PatientID').sum()
)

In [423]:
# All values >1 replaced by 1. 
mets_wide = (
    mets_wide.mask(mets_wide > 1, 1)
    .reset_index()
)

In [424]:
# Append missing training IDs.
mets_wide = (
    mets_wide.append(
        pd.Series(test_IDs)[~pd.Series(test_IDs).isin(mets_wide['PatientID'])].to_frame(name = 'PatientID'), 
        sort = False)
    .fillna(0)
)

In [425]:
row_ID(mets_wide)

(3786, 3786)

In [426]:
mets_wide.sample(5)

,PatientID,thorax_met,peritoneum_met,liver_met,other_gi_met,cns_met,bone_met,lymph_met,kidney_bladder_met,other_met
684,FB7252E516F54,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
528,F3CE7A1129925,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1254,FE7348C7058E6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2558,F1742665B1FEE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3463,FB1D4ED9A76A3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Merge

In [427]:
diagnosis_wide = pd.merge(diagnosis_elixhauser, other_cancer, on = 'PatientID')

In [428]:
diagnosis_wide = pd.merge(diagnosis_wide, mets_wide, on = 'PatientID')

In [429]:
row_ID(diagnosis_wide)

(3786, 3786)

In [430]:
list(diagnosis_wide.columns)

['PatientID',
 'chf',
 'cardiac_arrhythmias',
 'valvular_disease',
 'pulmonary_circulation',
 'peripheral_vascular',
 'htn_uncomplicated',
 'htn_complicated',
 'paralysis',
 'other_neuro_disorders',
 'chronic_pulmonary',
 'diabetes_uncomplicated',
 'diabetes_complicated',
 'hypothyroidism',
 'renal_failure',
 'liver_disease',
 'peptic_ulcer_disease',
 'aids_hiv',
 'lymphoma',
 'metastatic_cancer',
 'solid_tumor_wout_mets',
 'rheumatoid_arthritis',
 'coagulopathy',
 'obesity',
 'weight_loss',
 'fluid_electrolyte',
 'blood_loss_anemia',
 'deficiency_anemia',
 'alcohol_abuse',
 'drug_abuse',
 'psychoses',
 'depression',
 'elixhauser_other',
 'icd_count',
 'other_cancer',
 'thorax_met',
 'peritoneum_met',
 'liver_met',
 'other_gi_met',
 'cns_met',
 'bone_met',
 'lymph_met',
 'kidney_bladder_met',
 'other_met']

In [431]:
%whos DataFrame

Variable                 Type         Data/Info
-----------------------------------------------
biomarker_wide           DataFrame              PatientID brca_<...>\n[3786 rows x 2 columns]
cancer_10                DataFrame                PatientID Dia<...>\n[3224 rows x 3 columns]
cancer_9                 DataFrame                PatientID Dia<...>\n[1655 rows x 3 columns]
demographics             DataFrame              PatientID Gende<...>\n[3786 rows x 6 columns]
diagnosis                DataFrame                PatientID Dia<...>[124806 rows x 8 columns]
diagnosis_elix_10        DataFrame                PatientID Dia<...>[15741 rows x 35 columns]
diagnosis_elix_10_wide   DataFrame              PatientID  chf <...>n[2067 rows x 33 columns]
diagnosis_elix_9         DataFrame                PatientID Dia<...>n[7520 rows x 35 columns]
diagnosis_elix_9_wide    DataFrame              PatientID  chf <...>n[1337 rows x 33 columns]
diagnosis_elixhauser     DataFrame              PatientID 

In [432]:
# Keep biomarker_wide, demographics, diagnosis_wide, ecog_diagnosis_wide, enhanced_met, insurance_wide, 
# lab_wide, med_admin_wide, mortality, and weight_wide
del cancer_10
del cancer_9
del diagnosis
del diagnosis_elix_10
del diagnosis_elix_10_wide
del diagnosis_elix_9
del diagnosis_elix_9_wide
del diagnosis_elixhauser
del diagnosis_mets_10
del diagnosis_mets_9
del mets_wide
del other_cancer
del other_cancer_10
del other_cancer_9

### 11. Prior treatments 

#### Surgery or radiation 

In [433]:
prim_treat = pd.read_csv('Enhanced_MetPC_PrimaryTreatment.csv')

In [434]:
prim_treat = prim_treat[prim_treat['PatientID'].isin(test_IDs)]

In [435]:
row_ID(prim_treat)

(1616, 1616)

In [436]:
prim_treat.sample(5)

,PatientID,TreatmentType,TreatmentDate
165,FDB07FC21ACC0,Prostatectomy / Surgery,1999-01-01
63,F74EA5064C19D,Radiation Therapy (Including Brachytherapy and...,2011-11-01
1889,FE31760FE6FCE,Prostatectomy / Surgery,2015-02-03
7700,F0680EE17E2E2,Radiation Therapy (Including Brachytherapy and...,2013-12-05
3777,F0EE37CCAC25B,Radiation Therapy (Including Brachytherapy and...,2007-03-26


In [437]:
prim_treat.loc[:, 'TreatmentDate'] = pd.to_datetime(prim_treat['TreatmentDate'])

In [438]:
prim_treat = pd.merge(prim_treat, enhanced_met[['PatientID', 'met_date']], on = 'PatientID', how = 'left')

In [439]:
conditions = [
    (prim_treat['TreatmentType'] == 'Prostatectomy / Surgery'),
    (prim_treat['TreatmentType'] == 'Radiation Therapy (Including Brachytherapy and Radioactive Seeds)')]

choices = ['prostatectomy', 'radiation']

prim_treat.loc[:, 'prim_treatment'] = np.select(conditions, choices, default = 'other')

In [440]:
prim_treat = prim_treat[['PatientID', 'prim_treatment']]

In [441]:
# Append missing training IDs.
prim_treat_wide = (
    prim_treat.append(
        pd.Series(test_IDs)[~pd.Series(test_IDs).isin(prim_treat['PatientID'])].to_frame(name = 'PatientID'), 
        sort = False)
    .fillna('unknown')
)

In [442]:
row_ID(prim_treat_wide)

(3786, 3786)

#### ADT 

In [443]:
adt = pd.read_csv('Enhanced_MetPC_ADT.csv')

In [444]:
adt = adt[adt['PatientID'].isin(test_IDs)]

In [445]:
row_ID(adt)

(4080, 3659)

In [446]:
adt.sample(5)

,PatientID,TreatmentSetting,StartDate,EndDate,TreatmentStatus
4163,F09FEF7FB620A,Advanced,2017-06-08,NaN,Continuing
11015,F4246472EFBE8,Advanced,2014-06-01,NaN,Continuing
18970,F804CA54961B3,Advanced,2018-10-10,NaN,Continuing
5036,F204441C139EF,Advanced,2014-01-01,NaN,Continuing
472,F9E2BE23C1817,Curative,2006-08-11,2006-08-11,Discontinued


In [447]:
adt.loc[:, 'StartDate'] = pd.to_datetime(adt['StartDate'])

In [448]:
adt = pd.merge(adt, enhanced_met[['PatientID', 'met_date']], on = 'PatientID', how = 'left')

In [449]:
adt = (
    adt
    .assign(adt_diff = (adt['StartDate'] - adt['met_date']).dt.days)
)

In [450]:
adt = adt.query('adt_diff < -90')

In [451]:
row_ID(adt)

(1381, 1237)

In [452]:
adt = adt.drop_duplicates(subset = ['PatientID'], keep = 'first')

In [453]:
row_ID(adt)

(1237, 1237)

In [454]:
adt.loc[:,'early_adt'] = 1

In [455]:
adt = adt[['PatientID', 'early_adt']]

In [456]:
# Append missing training IDs.
adt_wide = (
    adt.append(
        pd.Series(test_IDs)[~pd.Series(test_IDs).isin(adt['PatientID'])].to_frame(name = 'PatientID'), 
        sort = False)
    .fillna(0)
)

In [457]:
row_ID(adt_wide)

(3786, 3786)

#### Merge

In [458]:
treat_wide = pd.merge(prim_treat_wide, adt_wide, on = 'PatientID')

In [459]:
row_ID(treat_wide)

(3786, 3786)

In [460]:
%whos DataFrame

Variable              Type         Data/Info
--------------------------------------------
adt                   DataFrame              PatientID  earl<...>\n[1237 rows x 2 columns]
adt_wide              DataFrame              PatientID  earl<...>\n[3786 rows x 2 columns]
biomarker_wide        DataFrame              PatientID brca_<...>\n[3786 rows x 2 columns]
demographics          DataFrame              PatientID Gende<...>\n[3786 rows x 6 columns]
diagnosis_wide        DataFrame              PatientID  chf <...>n[3786 rows x 44 columns]
ecog_diagnosis_wide   DataFrame               PatientID ecog<...>\n[3786 rows x 2 columns]
enhanced_met          DataFrame               PatientID diag<...>n[3786 rows x 17 columns]
insurance_wide        DataFrame              PatientID  medi<...>\n[3786 rows x 9 columns]
lab_wide              DataFrame              PatientID  albu<...>[3786 rows x 135 columns]
med_admin_wide        DataFrame              PatientID  ster<...>n[3786 rows x 13 columns]


In [461]:
# Keep biomarker_wide, demographics, ecog_diagnosis_wide, enhanced_met, mortality, 
# lab_wide, treat_wide, and weight_wide
del adt
del adt_wide
del prim_treat
del prim_treat_wide

## Part 3: File merge

In [462]:
enhanced_met = enhanced_met.drop(columns = ['diagnosis_date', 'met_date', 'CRPCDate'])

In [463]:
test_full = pd.merge(demographics, enhanced_met, on = 'PatientID')

In [464]:
test_full = pd.merge(test_full, mortality, on = 'PatientID')

In [465]:
test_full = pd.merge(test_full, med_admin_wide, on = 'PatientID')

In [466]:
test_full = pd.merge(test_full, biomarker_wide, on = 'PatientID')

In [467]:
test_full = pd.merge(test_full, insurance_wide, on = 'PatientID')

In [468]:
test_full = pd.merge(test_full, ecog_diagnosis_wide, on = 'PatientID')

In [469]:
test_full = pd.merge(test_full, weight_wide, on = 'PatientID')

In [470]:
test_full = pd.merge(test_full, lab_wide, on = 'PatientID')

In [471]:
test_full = pd.merge(test_full, diagnosis_wide, on = 'PatientID')

In [472]:
test_full = pd.merge(test_full, treat_wide, on = 'PatientID')

In [473]:
row_ID(test_full)

(3786, 3786)

In [474]:
len(test_full.columns)

228

In [475]:
list(test_full.columns)

['PatientID',
 'Gender',
 'race',
 'ethnicity',
 'age',
 'p_type',
 'NStage',
 'MStage',
 'Histology',
 'GleasonScore',
 'PSADiagnosis',
 'PSAMetDiagnosis',
 'stage',
 'met_year',
 'delta_met_diagnosis',
 'crpc',
 'crpc_time',
 'psa_diag_na',
 'psa_met_na',
 'death_status',
 'timerisk_activity',
 'steroid_diag',
 'opioid_PO_diag',
 'nonopioid_PO_diag',
 'pain_IV_diag',
 'ac_diag',
 'antiinfective_IV_diag',
 'antiinfective_diag',
 'antihyperglycemic_diag',
 'ppi_diag',
 'antidepressant_diag',
 'bta_diag',
 'thyroid_diag',
 'brca_status',
 'medicare',
 'medicaid',
 'medicare_medicaid',
 'commercial',
 'patient_assistance',
 'other_govt',
 'self_pay',
 'other',
 'ecog_diagnosis',
 'weight_diag',
 'bmi_diag',
 'bmi_diag_na',
 'weight_pct_change',
 'weight_pct_na',
 'weight_slope',
 'albumin_diag',
 'alp_diag',
 'alt_diag',
 'ast_diag',
 'bicarb_diag',
 'bun_diag',
 'calcium_diag',
 'chloride_diag',
 'creatinine_diag',
 'hemoglobin_diag',
 'neutrophil_count_diag',
 'platelet_diag',
 'potass

In [476]:
test_full.to_csv('test_full.csv', index = False, header = True)